<a href="https://colab.research.google.com/github/Cath-Strategic-Tech/adpdx_etl/blob/main/ADPDX_ClergyDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Introduction

The following notebook orchestrates the migration of ADPDX Accounts into Salesforce.


# Order of Operations

- Setup Enviro

  - [DONE] UDFs
  - [DONE] Load SF xref data

- ACCOUNTS

  - Extract Source Data
    - [DONE] Load 6 tables into separate dataframes
    - [DONE] Merge into single accounts table
    - DOING: Fix the ExternalID so that it references the original table, not the AccRecordType
  - Transform
    - TODO: Strip phone numbers
    - TODO: validate email addresses
    - TODO: handle churches that aren't parishes (missions, non-diocesan parishes, etc.)
  - Load
    - [DONE]Vicariates
    - [DONE] Organizations (Parishes, Schools, Newman Centres, Offices)
    - Religious
      - [DONE] Religious Parent accounts
      - [DONE] Religious Communities
      - [DONE] Religious Superiors (Contacts, set AccountID to Rel. Parent)
        - TODO: Handle invalid email addresses
        - TODO: Handle duplicate entries
      - TODO: Update Religious Communities with lookup to Rel. Superior

- CONTACTS

  - Extract

    - [DONE] Import Contact records
    - TODO: Get Photo directory @soames

  - Analysis

    - [DONE] Check columns & row count (3016)
    - [DONE] Identify unique languages

  - Transform

    - Complete ETL of fields that are more complex (search for TODO)
    - [DONE] Create new df_contact_staging, renaming columns to SF APIs
    - [DONE] Drop columns that don't map to Contact
    - Migrate Languages field (waiting on next package version) @soames
      - TODO: transform `,` to `;` so imports to multi-select list correctly
    - TODO: Concat Mailing Street Address lines into one
    - TODO: Handle Private Addresses: decide if will code changes or NOT use a custom Private Address field.
    - [DONE] Update boolean fields to True/False
    - [DONE] Set Contact Record Type (UDF)
    - [DONE] Validate, drop invalid emails
    - [DONE] Generate ExternalID > 'Archdpdx_External_Id\_\_c'
    - TODO: Preferred Email/Phone > where blank, set a default. Currently, all are getting set to 'Personal' and 'Mobile.'
    - TODO: Ecclesial Status (not mapping correctly)
    - [DONE] DROP columns that haven't been mapped yet

  - Load
    - [DONE] Set JobID to curr_job_id
    - [DONE] Handle character encoding that is geting messed up
    - TODO: Fix why the simple-salesforce insert isn't working

- CONTACTS > SPOUSES

  - TODO

- CONTACTS > PHOTOS

  - [DONE] Investigate how to migrate photos into a RTF (rich text field)
  - TODO

- CONTACTS > REGISTER ENTRIES

  - Parse columns into types of Sacraments or Notations
  - For lookups to Celebrants, query SF for contacts, create missing records
  - Generate External ID, apply to df
  - Clean up (remove extra columns, NaNs)
  - Upsert records

- CONTACTS > AFFILIATIONS

- AFFILIATIONS TABLE

  - Extract

    - [DONE] Turn the 'Org Table Name' & 'org Table Link' columns into External ID
    - Map in the Account IDs from SF
    -

  - Transform

    - Parse RecordTypeId
    - Parse Category
    - Map columns to SF field APIs

  - Load


# Setup Enviro


In [1]:
# !conda install -y simple-salesforce
# !conda install -y email_validator
# !conda install -y python-dotenv

# !conda install import-ipynb


In [2]:
# enviro setup

import pandas as pd

from datetime import datetime
now = datetime.now()

from simple_salesforce import Salesforce

In [3]:
# import environment variables (SF login credentials)
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
# @title Global Variables { run: "auto", vertical-output: true, display-mode: "both" }

target_enviro = "adpdx_devpro" # @param {type:"string"}

diocesan_account_id_devpro = "001Dx00001CwMTQIA3" # @param {type:"string"}

# @markdown The `run_upserts` variable controls whether or not upserts to Salesforce are executed when the notebook is run.
run_upserts = "True" # @param ["True", "False"]

In [5]:
# ADPDX dev_pro credentials
adpdx_devpro_user = os.getenv('ADPDX_DEVPRO_USER')
adpdx_devpro_pass = os.getenv('ADPDX_DEVPRO_PASS')
adpdx_devpro_token = os.getenv('ADPDX_DEVPRO_TOKEN')

# instantiate a SF session object
sf = Salesforce(domain='test', username='matt+adpdx@meribahflow.com.devpro', password=adpdx_devpro_pass, security_token=adpdx_devpro_token)

## UDFs


In [6]:
# Job ID Incrementer

def update_job_id(file_name):
    # Open the file in read mode and get the current job ID
    with open(file_name, 'r') as file:
        current_job_id = int(file.readline())

    # Increment the job ID
    new_job_id = current_job_id + 1

    # Open the file in write mode and update the job ID
    with open(file_name, 'w') as file:
        file.write(str(new_job_id))

    # Return the new job ID
    return new_job_id


# Concates two DF columns for an External ID

def concat_columns(df, columns, new_column, separator='_'):
    """
    Concatenates the values from specified columns into a single string
    with the specified separator and populates a new column in the DataFrame.

    Args:
    - df: pandas DataFrame
    - columns: list of column names to concatenate
    - new_column: name of the new column to be created
    - separator: separator to use between concatenated values (default is '_')

    Returns:
    - Updated pandas DataFrame with the new column
    """
    df[new_column] = df[columns].astype(str).apply(lambda x: separator.join(x), axis=1)
    return df



## Extract Salesforce xref data

The following cells downloads all records from the target Salesforce enviro for the following objects:

- RecordTypes
- Users
- Accounts
- Contacts


In [7]:
# get all ACTIVE SF users

sf_users = sf.query('Select Alias, FirstName, LastName, Username, id from User WHERE IsActive = True')
df_sf_users = pd.DataFrame(sf_users['records'])
df_sf_users = df_sf_users.drop(columns = 'attributes')
df_sf_users.shape

(8, 5)

In [8]:
# get all SF Record Types
get_all_recordTypes = 'Select Id, Name, DeveloperName, sObjecttype, namespaceprefix from RecordType'

# get list of records, add to dataframe
sf_recordTypes = sf.query(get_all_recordTypes)
df_sf_recordTypes = pd.DataFrame(sf_recordTypes['records'])
df_sf_recordTypes = df_sf_recordTypes.drop(columns = 'attributes')

# Create a dictionary mapping 'DeveloperName' to 'Id' for faster lookup
record_types_mapping = df_sf_recordTypes.set_index('DeveloperName')['Id'].to_dict()

df_sf_recordTypes

,Id,Name,DeveloperName,SobjectType,NamespacePrefix
0,012Dx0000007yCpIAI,Property,Property,Account,None
1,012Dx0000007yIOIAY,Ecclesial Affiliation,Ecclesial_Affiliation,mbfc__Placement__c,None
2,012Dx0000007yITIAY,Pastoral Assignments,Assignments_Clergy,mbfc__Placement__c,None
3,012Dx0000007yIYIAY,Staff,Staff,mbfc__Placement__c,None
4,012Dx0000007yIdIAI,Lay Person,Lay_Person,mbfc__Placement__c,None
5,012Dx0000007yOCIAY,Diocesan Appointment,Diocesan_Appointment,mbfc__Placement__c,None
6,012Dx0000007yOHIAY,Clergy/Religious Residence,Clergy_Religious_Residence,mbfc__Placement__c,None
7,012Dx0000007yOMIAY,Education,Education,mbfc__Placement__c,None
8,012Dx0000007yORIAY,Ministerial Status,Ministerial_Status,mbfc__Placement__c,None
9,012Dx0000007yTgIAI,z) All Types,All_Types,mbfc__Placement__c,None


In [9]:
# get SF Account
get_all_accounts = 'Select id, Name, RecordTypeId, Type, mbfc__Parish_Code__c, Job_Id__c, Archdpdx_Migration_Id__c from Account'

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')
df_sf_accounts.shape

(2000, 7)

In [10]:
# get SF Contacts
get_all_contacts = 'Select id, Name, npe01__Type_of_Account__c, RecordTypeId, Archdpdx_Migration_Id__c, CreatedById from Contact'

# get list of records, add to dataframe
sf_contacts = sf.query(get_all_contacts)
df_sf_contacts = pd.DataFrame(sf_contacts['records'])
df_sf_contacts = df_sf_contacts.drop(columns = 'attributes')
df_sf_contacts.shape

(2000, 6)

# ACCOUNTS


## Extract


### Load ArchdPDX csvs as DataFrames

ADPDX data for organizations is held in 6 tables, all of which will be migrated into Salesforce's Accounts object.


In [11]:
df_offices = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Offices.csv', skiprows= lambda x: x in [1])
df_offices["src_table"] = 'Offices'
df_offices["AccountRecordType"] = 'Organization'
df_offices.rename({"Name": "Account Name"}, axis="columns", inplace=True)
df_offices.columns

Index(['Record Number', 'Common Name', 'Account Name',
       'Archdiocese Assigns Clergy', 'Locator Description', 'Mailing Address',
       'Mailing Address 2', 'Mailing Address City', 'Mailing Address State',
       'Mailing Address Province', 'Mailing Address Postal Code',
       'Mailing Address Country', 'Phone', 'Fax', 'Email', 'Web Site',
       'src_table', 'AccountRecordType'],
      dtype='object')

In [12]:
df_parishes = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Parishes (3).csv', dtype={'Vicariate': 'object'}, skiprows= lambda x: x in [1])
df_parishes["src_table"] = 'Parishes'
df_parishes["AccountRecordType"] = 'Church'
df_parishes.rename({"Parish Formal Name": "Account Name"}, axis="columns", inplace=True)
df_parishes.columns

Index(['Record Number', 'Common Name', 'Sort Name', 'Parish Name',
       'Account Name', 'Parish City', 'Archdiocese Assigns Clergy',
       'Mission Of', 'Established', 'Vicariate', 'Non-Latin',
       'Locator Description', 'Mailing Address', 'Mailing Address 2',
       'Mailing Address City', 'Mailing Address State',
       'Mailing Address Province', 'Mailing Address Postal Code',
       'Mailing Address Country', 'County', 'Phone', 'Fax', 'Email',
       'Web Site', 'Disabled Access', 'Sanctuary Capacity',
       'Lat/Long Coordinates Decimal', 'Google Small Embed URL',
       'Miles to Pastoral Center', 'Schedule 1 Head', 'Schedule 1 Text',
       'Schedule 2 Head', 'Schedule 2 Text', 'Schedule 3 Head',
       'Schedule 3 Text', 'Schedule 4 Head', 'Schedule 4 Text',
       'Schedule 5 Head', 'Schedule 5 Text', 'Schedule 6 Head',
       'Schedule 6 Text', 'Schedule 7 Head', 'Schedule 7 Text', 'src_table',
       'AccountRecordType'],
      dtype='object')

In [13]:
df_religious = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/RelCommunities.csv', skiprows= lambda x: x in [1])
df_religious["src_table"] = 'RelCommunities'
df_religious["AccountRecordType"] = 'Religious'
df_religious.rename({"Community Name": "Account Name"}, axis="columns", inplace=True)
df_religious.columns

Index(['Record Number', 'Common Name', 'Account Name', 'Community City',
       'Archdiocese Assigns Clergy', 'Order Full Name', 'Order Common Name',
       'Order Letters', 'Men or Women', 'Non-Latin Rite', 'Show Order in Name',
       'Description', 'Locator Description', 'Mailing Address',
       'Mailing Address 2', 'Mailing Address City', 'Mailing Address State',
       'Mailing Address Province', 'Mailing Address Postal Code',
       'Mailing Address Country', 'Phone', 'Fax', 'Email', 'Web Site',
       'Religious Order', 'Secular Order', 'Diocesan Order',
       'Pontifical Order', 'Local Superior', 'Major Superior Name',
       'Major Superior Phone', 'Major Superior Email', 'src_table',
       'AccountRecordType'],
      dtype='object')

In [14]:
df_schools = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Schools.csv', skiprows= lambda x: x in [1])
df_schools["src_table"] = 'Schools'
df_schools["AccountRecordType"] = 'School'
df_schools.rename({"School Name": "Account Name"}, axis="columns", inplace=True)
df_schools.columns

Index(['Record Number', 'Common Name', 'Account Name', 'School City',
       'Archdiocese Assigns Clergy', 'Parish Link', 'Vicariate Link',
       'Archdiocesan School Code', 'Grades Provided', 'Established',
       'Locator Description', 'Mailing Address 1', 'Mailing Address 2',
       'Mailing Address City', 'Mailing Address State',
       'Mailing Address Province', 'Mailing Address Zip',
       'Mailing Address Country', 'Phone', 'Fax', 'Email', 'Web Site',
       'Lat/Long Coordinates Decimal', 'Google Small Embed URL', 'src_table',
       'AccountRecordType'],
      dtype='object')

In [15]:
df_vicariates = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Vicariates.csv', skiprows= lambda x: x in [1])
df_vicariates["src_table"] = 'Vicariates'
df_vicariates["AccountRecordType"] = 'Deanery'
# As we want to designate the Common Name as what will be the Account Name in Salesforce, we are renaming these columns in a different pattern than prior CSVs.
df_vicariates.rename({"Common Name": "Account Name"}, axis="columns", inplace=True)

df_vicariates.columns

Index(['Record Number', 'Account Name', 'Vicariate Name',
       'Archdiocese Assigns Clergy', 'src_table', 'AccountRecordType'],
      dtype='object')

In [16]:
df_newman = pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/NewmanCenters.csv', skiprows= lambda x: x in [1])
df_newman["src_table"] = 'NewmanCenters'
df_newman["AccountRecordType"] = 'Organization'
df_newman.rename({"Newman Center Name": "Account Name", "Newman Center City": "Mailing Address City2"}, axis="columns", inplace=True)
df_newman.columns

Index(['Record Number', 'Common Name', 'Account Name', 'Mailing Address City2',
       'Archdiocese Assigns Clergy', 'Established', 'Locator Description',
       'Mailing Address', 'Mailing Address 2', 'Mailing Address City',
       'Mailing Address State', 'Mailing Address Province',
       'Mailing Address Postal Code', 'Mailing Address Country', 'Phone',
       'Fax', 'Email', 'Web Site', 'Lat/Long Coordinates Decimal',
       'Google Small Embed URL', 'Miles to Pastoral Center', 'Schedule 1 Head',
       'Schedule 1 Text', 'Schedule 2 Head', 'Schedule 2 Text',
       'Schedule 3 Head', 'Schedule 3 Text', 'Schedule 4 Head',
       'Schedule 4 Text', 'Schedule 5 Head', 'Schedule 5 Text',
       'Schedule 6 Head', 'Schedule 6 Text', 'Schedule 7 Head',
       'Schedule 7 Text', 'src_table', 'AccountRecordType'],
      dtype='object')

Each of the 6 tables has an overlapping but distinct set of columns, making it challenging to conform these tables into a single staging table.

In addition, columns that correspond to the same field in salesforce are named differently in each table (eg. 'Parish City' vs. 'Religious City' vs. 'Newman Center City')


In [17]:
print('TABLE: (ROWS, COLUMNS)\n')

print(f'Offices:    {df_offices.shape}')
print(f'Parishes:   {df_parishes.shape}')
print(f'Religious:  {df_religious.shape}')
print(f'Schools:    {df_schools.shape}')
print(f'Vicariates: {df_vicariates.shape}')
print(f'Newman Ctr: {df_newman.shape}')

TABLE: (ROWS, COLUMNS)

Offices:    (35, 18)
Parishes:   (151, 45)
Religious:  (70, 34)
Schools:    (56, 26)
Vicariates: (18, 6)
Newman Ctr: (4, 37)


### Merge DFs into a single Accounts DF

This step takes 6 different tables and combines them into a single Accounts table for cleaning and staging.


In [18]:
# init list of DataFrames
src_accounts = [df_offices, df_parishes, df_religious, df_schools, df_vicariates, df_newman]

# concats the various Account dataframes into one large table
accounts = pd.concat(src_accounts, ignore_index=True)

accounts.head(5)

,Record Number,Common Name,Account Name,Archdiocese Assigns Clergy,Locator Description,Mailing Address,Mailing Address 2,Mailing Address City,Mailing Address State,Mailing Address Province,...,Major Superior Email,School City,Parish Link,Vicariate Link,Archdiocesan School Code,Grades Provided,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2
0,1,Pastoral Center,Pastoral Center,Yes,NaN,2838 E Burnside St,NaN,Portland,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Catholic Sentinel,Catholic Sentinel,No,NaN,2838 E Burnside St,NaN,Portland,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Catholic Cemeteries,Catholic Cemeteries,No,NaN,333 SW Skyline Blvd,NaN,Portland,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,Griffin Center,Griffin Center,No,NaN,11957 SE Fuller Rd,NaN,Milwaukie,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,Providence Portland Medical Center,Providence Portland Medical Center,Yes,NaN,4805 NE Glisan St,NaN,Portland,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Time to do some table column renaming and re-organizing!


In [19]:
# renames columns headers to consolidate account names into SF-conformed data model
accounts.rename({"Common Name": "Name, City"}, axis="columns", inplace=True)

accounts.rename(
    columns={
        'Account Name': 'Name',
        'Mailing Address': 'BillingStreet',
        'Mailing Address 2': 'BillingStreet2',
        'Mailing Address City': 'BillingCity',
        'Mailing Address State': 'BillingState',
        'Mailing Address Postal Code': 'BillingPostalCode',
        'Mailing Address Country': 'BillingCountry',
        'Email': 'mbfc__Email__c',
        'Web Site': 'Website',
        'Order Common Name': 'mbfc__Abbreviation__c',
        'Order Letters': 'mbfc__Religious_Suffix__c',
        'Men or Women': 'mbfc__Type_Members__c'
    },
    inplace=True
)

# reorder column order
col = accounts.pop('Name')
accounts.insert(2, col.name, col)

col = accounts.pop('Parish Name')
accounts.insert(3, col.name, col)

col = accounts.pop('AccountRecordType')
accounts.insert(1, col.name, col)

accounts[accounts.BillingStreet2.isna() == False]

,Record Number,AccountRecordType,"Name, City",Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,BillingStreet2,BillingCity,...,Major Superior Email,School City,Parish Link,Vicariate Link,Archdiocesan School Code,Grades Provided,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2
14,32,Organization,Diaconate Office,Diaconate Office,NaN,Yes,NaN,Pastoral Center,2838 E Burnside St,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,58,Organization,Office of Marketing and Communications,Office of Marketing and Communications,NaN,Yes,NaN,Pastoral Center,2838 E Burnside St,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1,Church,"Our Lady of Perpetual Help, St Mary’s, Albany","Our Lady of Perpetual Help, St Mary’s",NaN,Yes,SW Ellsworth St between 8th and 9th Streets,"Our Lady of Perpetual Help, St Mary’s Parish",815 Broadalbin St SW,Albany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2,Church,"St. Andrew Dũng-Lạc Mission, Aloha",St. Andrew Dũng-Lạc,NaN,No,SW Grabhorn Rd/209th Ave and Farmington Rd,St. Andrew Dũng-Lạc Mission,7390 SW Grabhorn Rd,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,3,Church,"St. Elizabeth Ann Seton, Aloha",St. Elizabeth Ann Seton,NaN,Yes,NaN,St. Elizabeth Ann Seton Parish,3145 SW 192nd Ave,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,62,Religious,"Work of Jesus the High Priest, Gresham (OJSS)",Work of Jesus the High Priest,NaN,No,NaN,OJSS Community,451 NW 1st St,Gresham,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,64,Religious,"Heralds of the Good News, Portland (HGN)",Heralds of the Good News,NaN,No,NaN,c/o Chancellor,2838 E Burnside St,Portland,...,rkappumkal@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,65,Religious,"Missionary Oblates of Mary Immaculate, Rome, I...",Missionary Oblates of Mary Immaculate,NaN,No,NaN,Missionary Oblates of Mary Immaculate,Via Aurelia 290,Roma,...,gensec@omigen.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,73,Religious,"Brothers of Saint John, Laredo, TX (CSJ)",Brothers of Saint John,NaN,No,NaN,St. John Priory,505 Century Dr S,Laredo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# export merged tables DESCRIPTION to CSV for mapping
accounts.describe(include='all').transpose().to_csv(f'/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/working/accounts.csv')
accounts.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Record Number,334.0,NaN,NaN,NaN,54.5,41.389801,1.0,21.25,45.0,76.75,173.0
AccountRecordType,334,5,Church,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Name, City",316,316,Pastoral Center,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,334,291,St. Mary,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parish Name,5,5,St. Anne,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Grades Provided,52,12,PS-8,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mailing Address 1,56,55,4420 SW St Marys Dr,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mailing Address Zip,56.0,NaN,NaN,NaN,97222.446429,124.9586,97005.0,97134.75,97217.5,97301.0,97526.0
Vicariate Name,18,18,Albany-Corvallis,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transform


In [21]:
# Create a single BillingAddress field
# billingstreet = str(f"Mailing Addresss 1 /n Mailing Address 2")
# accounts['BillingStreet'] = accounts['Mailing Address 1'].astype(str) + accounts['Mailing Address 2'].astype(str)

### AccountRecordType & ChurchType


In [22]:
# Sets all rows where AccountRecordType is Church as a Parish. THIS MAY NEED NUANCING
accounts.loc[accounts['AccountRecordType'] == 'Church', 'mbfc__Church_Type__c'] = 'Parish'
accounts[accounts['AccountRecordType'] == 'Church'].head(5)


,Record Number,AccountRecordType,"Name, City",Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,BillingStreet2,BillingCity,...,School City,Parish Link,Vicariate Link,Archdiocesan School Code,Grades Provided,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2,mbfc__Church_Type__c
35,1,Church,"Our Lady of Perpetual Help, St Mary’s, Albany","Our Lady of Perpetual Help, St Mary’s",NaN,Yes,SW Ellsworth St between 8th and 9th Streets,"Our Lady of Perpetual Help, St Mary’s Parish",815 Broadalbin St SW,Albany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
36,2,Church,"St. Andrew Dũng-Lạc Mission, Aloha",St. Andrew Dũng-Lạc,NaN,No,SW Grabhorn Rd/209th Ave and Farmington Rd,St. Andrew Dũng-Lạc Mission,7390 SW Grabhorn Rd,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
37,3,Church,"St. Elizabeth Ann Seton, Aloha",St. Elizabeth Ann Seton,NaN,Yes,NaN,St. Elizabeth Ann Seton Parish,3145 SW 192nd Ave,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
38,4,Church,"St. Peter the Fisherman Mission, Arch Cape",St. Peter the Fisherman,NaN,Yes,79441 Hwy 101 S,St. Peter the Fisherman Mission,PO Box 29,Seaside,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
39,5,Church,"Our Lady of the Mountain, Ashland",Our Lady of the Mountain,NaN,Yes,NaN,Our Lady of the Mountain Parish,987 Hillview Dr,Ashland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish


### Generate ExternalId


In [23]:
# Generate an External ID
# columns_to_concate = ['AccountRecordType', 'Record Number']
# accounts = concat_columns(accounts, columns_to_concate, 'Archdpdx_Migration_Id__c', separator='_')

In [24]:
# NEW Generate an External ID
columns_to_concate = ['src_table', 'Record Number']
accounts = concat_columns(accounts, columns_to_concate, 'Archdpdx_Migration_Id__c', separator='_')

In [25]:
# set Deanery RecordTypeId to the Church RecordTypeId
# map in RecordTypeIds
accounts['RecordTypeId'] = accounts['AccountRecordType'].map(record_types_mapping)
record_types_mapping

{'Property': '012Dx0000007yCpIAI',
 'Ecclesial_Affiliation': '012Dx0000007yIOIAY',
 'Assignments_Clergy': '012Dx0000007yITIAY',
 'Staff': '012Dx0000007yIYIAY',
 'Lay_Person': '012Dx0000009TK3IAM',
 'Diocesan_Appointment': '012Dx0000007yOCIAY',
 'Clergy_Religious_Residence': '012Dx0000007yOHIAY',
 'Education': '012Dx0000009TKBIA2',
 'Ministerial_Status': '012Dx0000007yORIAY',
 'All_Types': '012Dx0000007yTgIAI',
 'Church': '012Dx0000009TJxIAM',
 'Deanery': '012Dx0000009TJyIAM',
 'Group': '012Dx0000009TJzIAM',
 'Religious': '012Dx0000009TK6IAM',
 'School': '012Dx0000009TK1IAM',
 'Consecrated': '012Dx0000009TK2IAM',
 'Permanent_Deacon': '012Dx0000009TK4IAM',
 'Priest': '012Dx0000009TK5IAM',
 'All': '012Dx0000009TK7IAM',
 'Chancery_Users': '012Dx0000009TK8IAM',
 'Diocean_Users': '012Dx0000009TK9IAM',
 'Parish_Users': '012Dx0000009TKAIA2',
 'Employment': '012Dx0000009TKCIA2',
 'Ministry_Volunteer': '012Dx0000009TKDIA2',
 'Parishioner': '012Dx0000009TKEIA2',
 'MajorGift': '012Hu000001pkqBIAQ'

In [26]:
#TODO: Set 'organization type' field value for each account in the 'organization' load: Offices, Newman Centres, Schools, Organizations
# Might be best if it is set manually at the end of the migration.

## Load


### Generate a new Job ID


In [27]:
# increment to the job_id
file_name = '/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/jobs/job_id'
curr_job_id = update_job_id(file_name)
print(f"New job ID: {curr_job_id}")

# add/update account DF with job_id
accounts["Job_Id__c"] = curr_job_id


New job ID: 49


### A) Vicariates


In [28]:
vicariates = accounts[accounts['AccountRecordType'] == 'Deanery']

vicariates = vicariates[[
    'Record Number',
    'Name',
    # 'AccountRecordType',
    'Job_Id__c',
    'Archdpdx_Migration_Id__c',
    'RecordTypeId'
    ]]

# add parentid
vicariates["mbfc__Diocese__c"] = diocesan_account_id_devpro
vicariates['ParentId'] = diocesan_account_id_devpro
vicariates['mbfc__Church_Type__c'] = 'Deanery'

vicariates.rename(columns={
        # 'Name, City': 'Name',
        'External_Id': 'Archdpdx_Migration_Id__c'
    }, inplace=True)

vicariates.reset_index()
vicariates.set_index('Record Number', inplace=True)

vicariates

,Name,Job_Id__c,Archdpdx_Migration_Id__c,RecordTypeId,mbfc__Diocese__c,ParentId,mbfc__Church_Type__c
Record Number,,,,,,,
1,Albany-Corvallis Vicariate,49,Vicariates_1,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
2,"Beaverton, Suburban Vicariate",49,Vicariates_2,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
3,Columbia County Vicariate,49,Vicariates_3,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
4,Downtown Portland Vicariate,49,Vicariates_4,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
5,"East Portland, Suburban Vicariate",49,Vicariates_5,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
6,Marion County Vicariate,49,Vicariates_6,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
7,Metropolitan Eugene Vicariate,49,Vicariates_7,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
8,Metropolitan Salem Vicariate,49,Vicariates_8,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
9,North Coast Vicariate,49,Vicariates_9,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery


#### Export Vicariates to CSV


In [29]:
# export to CSV
vicariates.to_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/vicariates_staging.csv')


#### Upsert Vicariates


In [30]:
bulk_data = []
for row in vicariates.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    vicariate_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    upserts = pd.DataFrame(vicariate_upsert)

    print(upserts)
    

    success  created                  id errors
0      True    False  001Dx00001FaTb0IAF     []
1      True    False  001Dx00001FaTb1IAF     []
2      True    False  001Dx00001FaTb2IAF     []
3      True    False  001Dx00001FaTb3IAF     []
4      True    False  001Dx00001FaTb4IAF     []
5      True    False  001Dx00001FaTb5IAF     []
6      True    False  001Dx00001FaTb6IAF     []
7      True    False  001Dx00001FaTb7IAF     []
8      True    False  001Dx00001FaTb8IAF     []
9      True    False  001Dx00001FaTb9IAF     []
10     True    False  001Dx00001FaTbAIAV     []
11     True    False  001Dx00001FaTbBIAV     []
12     True    False  001Dx00001FaTbCIAV     []
13     True    False  001Dx00001FaTbDIAV     []
14     True    False  001Dx00001FaTbEIAV     []
15     True    False  001Dx00001FaTbFIAV     []
16     True    False  001Dx00001FaTbGIAV     []
17     True    False  001Dx00001FaTbHIAV     []


In [31]:
# create a map of Vicariate lookup ids to the unique ids generated on the Vicariate records
# vicariates_externalid_map = vicariates.set_index('Record Number')['Archdpdx_Migration_Id__c'].to_dict()

# vicariates_externalid_map

In [32]:
# Generate an Errors log
import csv

keys = vicariate_upsert[0].keys()

with open('vicariate_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(vicariate_upsert)

In [33]:
# @title Get Vicariate records from SF

sf_deaneries = sf.query("SELECT Archdpdx_Migration_Id__c, Id FROM Account WHERE RecordType.DeveloperName = 'Deanery'")

df_sf_deaneries = pd.DataFrame(sf_deaneries['records'])
df_sf_deaneries = df_sf_deaneries.drop(columns = 'attributes')

df_sf_deaneries

# Creates a dict of Vicariate unique ids to the new Salesforce record IDs, so can populate on latter Account records
vicariate_sf_recordids = df_sf_deaneries.set_index('Archdpdx_Migration_Id__c')['Id'].to_dict()
vicariate_sf_recordids

{'Vicariates_1': '001Dx00001FaTb0IAF',
 'Vicariates_2': '001Dx00001FaTb1IAF',
 'Vicariates_3': '001Dx00001FaTb2IAF',
 'Vicariates_4': '001Dx00001FaTb3IAF',
 'Vicariates_5': '001Dx00001FaTb4IAF',
 'Vicariates_6': '001Dx00001FaTb5IAF',
 'Vicariates_7': '001Dx00001FaTb6IAF',
 'Vicariates_8': '001Dx00001FaTb7IAF',
 'Vicariates_9': '001Dx00001FaTb8IAF',
 'Vicariates_10': '001Dx00001FaTb9IAF',
 'Vicariates_11': '001Dx00001FaTbAIAV',
 'Vicariates_12': '001Dx00001FaTbBIAV',
 'Vicariates_13': '001Dx00001FaTbCIAV',
 'Vicariates_14': '001Dx00001FaTbDIAV',
 'Vicariates_15': '001Dx00001FaTbEIAV',
 'Vicariates_16': '001Dx00001FaTbFIAV',
 'Vicariates_17': '001Dx00001FaTbGIAV',
 'Vicariates_18': '001Dx00001FaTbHIAV',
 'Deanery_1': '001Dx00001CwdOmIAJ',
 'Deanery_2': '001Dx00001CwdOnIAJ'}

### B) Parishes, Schools, Organizations


In [34]:
# NEW Create a new DF with Account records - excluding Deaneries (already handled) and Religious (to be handled differently, after)
acc_main = accounts[accounts['AccountRecordType'] != 'Deanery']
acc_main = acc_main[acc_main['AccountRecordType'] != 'Religious']

acc_main.loc[acc_main['AccountRecordType'] == 'Church', 'Vicariate_Ext_Id'] = 'Vicariates_' + acc_main['Vicariate']

acc_main.sample(5)

,Record Number,AccountRecordType,"Name, City",Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,BillingStreet2,BillingCity,...,Grades Provided,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2,mbfc__Church_Type__c,Archdpdx_Migration_Id__c,RecordTypeId,Job_Id__c,Vicariate_Ext_Id
299,46,School,"Holy Redeemer Catholic School, Portland",Holy Redeemer Catholic School,NaN,Yes,NaN,NaN,NaN,Portland,...,PS-8,127 N Rosa Parks Way,97217.0,NaN,NaN,NaN,Schools_46,012Dx0000009TK1IAM,49,NaN
69,39,Church,"Holy Family Mission, Glendale",Holy Family,NaN,Yes,243 Marshall St,Holy Family Mission,PO Box 810,Myrtle Creek,...,NaN,NaN,NaN,NaN,NaN,Parish,Parishes_39,012Dx0000009TJxIAM,49,Vicariates_15
185,173,Church,"St. Thomas More Newman Center, Eugene",St. Thomas More Newman Center,NaN,No,NaN,St. Thomas More Newman Center Parish,1850 Emerald St,Eugene,...,NaN,NaN,NaN,NaN,NaN,Parish,Parishes_173,012Dx0000009TJxIAM,49,Vicariates_7
143,114,Church,"St. Thomas More, Portland",St. Thomas More,NaN,Yes,SW Patton Rd and Humphrey Blvd,St. Thomas More Parish,3525 SW Patton Rd,Portland,...,NaN,NaN,NaN,NaN,NaN,Parish,Parishes_114,012Dx0000009TJxIAM,49,Vicariates_4
31,57,Organization,Providence Newberg Medical Center Pastoral Care,Providence Newberg Medical Center Pastoral Care,NaN,No,NaN,1001 Providence Drive,NaN,Newberg,...,NaN,NaN,NaN,NaN,NaN,NaN,Offices_57,012Hu000001pkqEIAQ,49,NaN


In [35]:
# OLD > Create a new DF with Account records - excluding Deaneries (already handled) and Religious (to be handled differently, after)
# acc_main = accounts[accounts['AccountRecordType'] != 'Deanery']
# acc_main = acc_main[acc_main['AccountRecordType'] != 'Religious']

# acc_main.loc[acc_main['AccountRecordType'] == 'Church', 'Vicariate_Ext_Id'] = 'Deanery_' + acc_main['Vicariate']

# acc_main.sample(5)

In [36]:
# TODO: This is now obsolete (I think)
# acc_main['VicariateUniqueId'] = acc_main['Vicariate'].map(vicariates_externalid_map)

acc_main['mbfc__Deanery__c'] = acc_main.Vicariate_Ext_Id.map(vicariate_sf_recordids)

acc_main[acc_main['AccountRecordType'] == 'Church']['mbfc__Deanery__c']

35     001Dx00001FaTb0IAF
36     001Dx00001FaTbCIAV
37     001Dx00001FaTbFIAV
38     001Dx00001FaTb8IAF
39     001Dx00001FaTbEIAV
              ...        
181    001Dx00001FaTb4IAF
182    001Dx00001FaTbGIAV
183    001Dx00001FaTb5IAF
184    001Dx00001FaTbHIAV
185    001Dx00001FaTb6IAF
Name: mbfc__Deanery__c, Length: 151, dtype: object

In [37]:
# Clean up NaN values

acc_main.fillna('', inplace=True)

In [38]:
# @title Export to CSV
# Export to CSV for manual loading

accounts_staging = acc_main[[
    'Name',
    'RecordTypeId',
    'mbfc__Church_Type__c',
    'mbfc__Deanery__c',
    'BillingStreet',
    'BillingCity',
    'BillingState',
    'BillingPostalCode',
    'BillingCountry',
    'Phone',
    'Fax',
    'mbfc__Email__c',
    'Website',
    'mbfc__Abbreviation__c',
    'mbfc__Religious_Suffix__c',
    'mbfc__Type_Members__c',
    'Description',
    'Job_Id__c',
    'Archdpdx_Migration_Id__c'

    ]]

In [39]:
accounts_staging.to_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/accounts_staging.csv', encoding='utf-8-sig')

#### Upsert Accounts (TBD )


In [40]:
bulk_data = []
for row in accounts_staging.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

In [41]:
if run_upserts == 'True':
    account_staging_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    account_upserts = pd.DataFrame(account_staging_upsert)

account_upserts

,success,created,id,errors
0,True,False,001Dx00001FaTcBIAV,[]
1,True,False,001Dx00001FaTcCIAV,[]
2,True,False,001Dx00001FaTcDIAV,[]
3,True,False,001Dx00001FaTcEIAV,[]
4,True,False,001Dx00001FaTcFIAV,[]
...,...,...,...,...
241,True,False,001Dx00001FaTg4IAF,[]
242,True,False,001Dx00001FaTg5IAF,[]
243,True,False,001Dx00001FaTg6IAF,[]
244,True,False,001Dx00001FaTg7IAF,[]


In [42]:
# Generate an Errors log
import csv

keys = account_staging_upsert[0].keys()

with open('accounts_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(account_staging_upsert)

# TODO: Convert this into a UDF

In [43]:
# @title Extract SF Account records

sf_accounts = sf.query('Select id, Name, RecordTypeId, mbfc__Church_Type__c, Archdpdx_Migration_Id__c, Job_Id__c from Account WHERE Job_Id__c != null')
sf_accounts = pd.DataFrame(sf_accounts['records'])
sf_accounts = sf_accounts.drop(columns = 'attributes')
sf_accounts

,Id,Name,RecordTypeId,mbfc__Church_Type__c,Archdpdx_Migration_Id__c,Job_Id__c
0,001Dx00001DOFt5IAH,"Our Lady of Perpetual Help Church, St Mary's, ...",012Dx0000009TJxIAM,Church Building,Church_1_cloned_by_sl,12
1,001Dx00001FZm6kIAD,St. Francis of Assisi School,012Hu000001pkqEIAQ,None,Organization_5,36
2,001Dx00001FZm6lIAD,Valley Catholic Elementary School,012Hu000001pkqEIAQ,None,Organization_8,36
3,001Dx00001FZm6mIAD,Valley Catholic Middle School,012Hu000001pkqEIAQ,None,Organization_9,36
4,001Dx00001FZm6nIAD,Valley Catholic High School,012Hu000001pkqEIAQ,None,Organization_10,36
...,...,...,...,...,...,...
373,001Dx00001FaTg4IAF,Resurrection Catholic Parish School,012Dx0000009TK1IAM,None,Schools_58,49
374,001Dx00001FaTg5IAF,OSU Newman Center,012Hu000001pkqEIAQ,None,NewmanCenters_1,49
375,001Dx00001FaTg6IAF,St. Thomas More (UO) Newman Center,012Hu000001pkqEIAQ,None,NewmanCenters_2,49
376,001Dx00001FaTg7IAF,Walsh Memorial (SOU) Newman Center at Our Lady...,012Hu000001pkqEIAQ,None,NewmanCenters_3,49


### C) Religious Institutes (Parents)


In [44]:
"""
- 'acc_religious' DF: create unique_id of religious parents
- create 'acc_religious_orders' DF , upsert into SF
- extract accounts from Salesforce, create dict (external_ID : account_ID)
- map parent ids onto religious child accounts DF in main DF
- 'acc_religious' > staging DF ('acc_religious')
    - drop unnecessary columns
    - upsert create DF of religious children, upsert into SF with
"""

# Create a new DF of all Religious accounts
acc_religious = accounts[accounts['AccountRecordType'] == 'Religious']

# Create a simplified external ID field
acc_religious['Archdpdx_Migration_Id__c'] = acc_religious['Order Full Name'].apply(
    lambda x: x.lower().replace(' ', '')[:40]
)

acc_religious_2 = acc_religious

# Create a DF for only parent religious order accounts
acc_religious_parents = acc_religious_2[['Order Full Name', 'Name', 'mbfc__Abbreviation__c', 'mbfc__Religious_Suffix__c', 'mbfc__Type_Members__c', 'Archdpdx_Migration_Id__c']]

# Drop duplicate rows of the same parent Religious Order (becuase there are more than 1 local community of a particular order)
acc_religious_parents.drop_duplicates('Order Full Name', inplace=True)

# How many remaining rows after dropping duplicates?
print(acc_religious_parents.shape)

# Rename columns
acc_religious_parents = acc_religious_parents.rename(columns={
    'Order Full Name': 'Description'
    })

# Drop NA
acc_religious_parents.fillna('', inplace=True)

acc_religious_parents


(62, 6)


/var/folders/_g/p51k4k2d5fg45slnqjhgd3sc0000gn/T/ipykernel_47420/1080047542.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_religious['Archdpdx_Migration_Id__c'] = acc_religious['Order Full Name'].apply(
/var/folders/_g/p51k4k2d5fg45slnqjhgd3sc0000gn/T/ipykernel_47420/1080047542.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_religious_parents.drop_duplicates('Order Full Name', inplace=True)


,Description,Name,mbfc__Abbreviation__c,mbfc__Religious_Suffix__c,mbfc__Type_Members__c,Archdpdx_Migration_Id__c
186,Societas Iesu,Colombiere Jesuit Community,Jesuits,SJ,Men,societasiesu
187,Ordo Cisterciensis Strictioris Observantiae,Abbey of Our Lady of Guadalupe,Trappists,OCSO,Men,ordocisterciensisstrictiorisobservantiae
189,Ordo Sancti Benedicti,Benedictine Monks of Mount Angel Abbey,Benedictines,OSB,Men,ordosanctibenedicti
190,Misioneros del Espíritu Santo,Missionaries of the Holy Spirit Provincial House,"Missionaries of the Holy Spirit, Christ the Pr...",MSpS,Men,misionerosdelespíritusanto
191,Apostles of Jesus,Apostles of Jesus,Apostles of Jesus,AJ,Men,apostlesofjesus
...,...,...,...,...,...,...
249,Fraternità san Carlo Borromeo,Priestly Fraternity of the Missionaries of St....,Fraternity of St. Charles,FSCB,Men,fraternitàsancarloborromeo
250,"Sons of Mary, Mother of Mercy","Sons of Mary, Mother of Mercy","Sons of Mary, Mother of Mercy",SMMM,Men,"sonsofmary,motherofmercy"
251,Society of the Divine Word,Society of the Divine Word,Society of the Divine Word,SVD,Men,societyofthedivineword
252,Society of the Divine Saviour,Society of the Divine Saviour,Society of the Divine Saviour,SDS,Men,societyofthedivinesaviour


In [45]:
acc_religious_parents['Religious_Type__c'] = 'Congregation'

In [46]:
#TODO: 'Religious Order', 'Secular Order', 'Diocesan Order', 'Pontifical Order'

In [47]:
# @title  Set recordType to 'Religious'

#
religious_recordtype_id = df_sf_recordTypes.loc[
    (df_sf_recordTypes['DeveloperName'] == 'Religious') & (df_sf_recordTypes['SobjectType'] == 'Account'),
    'Id'
    ].iloc[0]  # Use .iloc[0] to get the first item if you're expecting exactly one match

print(religious_recordtype_id)

acc_religious_parents['RecordTypeId'] = religious_recordtype_id

acc_religious_parents.sample(10)

012Dx0000009TK0IAM


,Description,Name,mbfc__Abbreviation__c,mbfc__Religious_Suffix__c,mbfc__Type_Members__c,Archdpdx_Migration_Id__c,Religious_Type__c,RecordTypeId
224,"Sisters of Providence, Mother Joseph Province",Sisters of Providence,Sisters of Providence,SP,Women,"sistersofprovidence,motherjosephprovince",Congregation,012Dx0000009TK0IAM
191,Apostles of Jesus,Apostles of Jesus,Apostles of Jesus,AJ,Men,apostlesofjesus,Congregation,012Dx0000009TK0IAM
200,Domus Dei Clerical Society of Apostolic Life,Society of Domus Dei Holy House Monasteries,Domus Dei,SDD,Men,domusdeiclericalsocietyofapostoliclife,Congregation,012Dx0000009TK0IAM
197,Brotherhood of the People of Praise,Brotherhood of the People of Praise,Brotherhood of the People of Praise,,Men,brotherhoodofthepeopleofpraise,Congregation,012Dx0000009TK0IAM
236,Opera di Gesù Sommo Sacerdote,Work of Jesus the High Priest,Work of Jesus the High Priest,OJSS,Men,operadigesùsommosacerdote,Congregation,012Dx0000009TK0IAM
230,Sociedad de María,Society of Mary,Society of Mary,SdM,Women,sociedaddemaría,Congregation,012Dx0000009TK0IAM
218,Sisters of Charity of the Blessed Virgin Mary,Sisters of Charity of the Blessed Virgin Mary,Sisters of Charity of the Blessed Virgin Mary,BVM,Women,sistersofcharityoftheblessedvirginmary,Congregation,012Dx0000009TK0IAM
228,Congregation of the Sisters of St. Joseph of P...,Sisters of St. Joseph of Peace,Sisters of St. Joseph,CSJP,Women,congregationofthesistersofst.josephofpea,Congregation,012Dx0000009TK0IAM
253,Society of Our Lady of the Most Holy Trinity,Society of Our Lady of the Most Holy Trinity,Society of Our Lady of the Most Holy Trinity,SOLT,Men,societyofourladyofthemostholytrinity,Congregation,012Dx0000009TK0IAM
227,Sisters of St. Francis of Philadelphia,Sisters of St. Francis of Philadelphia,Sisters of St. Francis of Philadelphia,OSF,Women,sistersofst.francisofphiladelphia,Congregation,012Dx0000009TK0IAM


In [48]:
# @title Send to CSV
acc_religious_parents.to_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/religious_order_staging.csv', encoding='utf-8-sig')

In [49]:
# @title Upsert to Salesforce
bulk_data = []
for row in acc_religious_parents.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    religious_order_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_order_upsert = pd.DataFrame(religious_order_upsert)

df_rel_order_upsert

,success,created,id,errors
0,True,False,001Dx00001FZmZjIAL,[]
1,True,False,001Dx00001FZmZkIAL,[]
2,True,False,001Dx00001FZmZlIAL,[]
3,True,False,001Dx00001FZmb9IAD,[]
4,True,False,001Dx00001FZmZmIAL,[]
...,...,...,...,...
57,True,False,001Dx00001FZmabIAD,[]
58,True,False,001Dx00001FZmacIAD,[]
59,True,False,001Dx00001FZmadIAD,[]
60,True,False,001Dx00001FZmaeIAD,[]


In [50]:
# Generate an Errors log
import csv

keys = religious_order_upsert[0].keys()

with open('religious_order_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(religious_order_upsert)

# TODO: Convert this into a UDF

In [51]:
# @title get SF Accounts
get_all_rel_accounts = f"Select id, Name, RecordTypeId, Type, Archdpdx_Migration_Id__c from Account where RecordTypeID = '{religious_recordtype_id}'"

print(religious_recordtype_id)

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_rel_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')

df_sf_accounts.sample(10)

012Dx0000009TK0IAM


,Id,Name,RecordTypeId,Type,Archdpdx_Migration_Id__c
29,001Dx00001FZmZlIAL,Benedictine Monks of Mount Angel Abbey,012Dx0000009TK0IAM,None,ordosanctibenedicti
40,001Dx00001FZmZxIAL,Servite Friars,012Dx0000009TK0IAM,None,ordoservorumbeataemariaevirginis
45,001Dx00001FZma2IAD,Sisters of Our Lady of Sorrows,012Dx0000009TK0IAM,None,franciscanmissionarysistersofourladyofso
32,001Dx00001FZmZoIAL,Apostolic Life Community,012Dx0000009TK0IAM,None,apostoliclifecommunityofpriestsintheopus
120,001Dx00001FaTiYIAV,"Adrian Dominican Sisters, Adrian, MI (OP)",012Dx0000009TK0IAM,None,RelCommunities_30
86,001Dx00001FaTiuIAF,"Sisters of St. Joseph of Peace, Eugene (CSJP)",012Dx0000009TK0IAM,None,RelCommunities_52
66,001Dx00001FZmaMIAT,Sacred Heart Maronite Monastery,012Dx0000009TK0IAM,None,maronitemonksofjesusmaryandjoseph
37,001Dx00001FZmZuIAL,Franciscan Friars,012Dx0000009TK0IAM,None,ordofratrumminorumprovinceofsaintbarbara
57,001Dx00001FZmaDIAT,Sisters of Mercy of the Americas,012Dx0000009TK0IAM,None,sistersofmercyoftheamericaswest/midwestr
72,001Dx00001FZmaSIAT,"Order of Friars Minor, Capuchins",012Dx0000009TK0IAM,None,"orderoffriarsminor,capuchins"


In [52]:
religious_order_mapping = df_sf_accounts.set_index('Archdpdx_Migration_Id__c')['Id'].to_dict()
# religious_order_mapping

### D) Religious Communities


In [53]:
acc_religious_staging = (acc_religious
                         .rename(columns={'Archdpdx_Migration_Id__c' : 'Parent_Archdpdx_Migration_Id__c'})
)

acc_religious_staging['ParentId'] = acc_religious_staging['Parent_Archdpdx_Migration_Id__c'].map(religious_order_mapping)

In [54]:
# Enrich the data

acc_religious_staging['Religious_Type__c'] = 'Local Community'
acc_religious_staging['Archdpdx_Migration_Id__c'] = 'RelCommunities_' + acc_religious_staging['Record Number'].astype('str')
acc_religious_staging['RecordTypeId'] = religious_recordtype_id
acc_religious_staging.drop(columns='Name', inplace=True)
acc_religious_staging.rename(columns={
    'Name, City': 'Name'
}, inplace=True)

acc_religious_staging.sample(5)

,Record Number,AccountRecordType,Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,BillingStreet2,BillingCity,BillingState,...,Mailing Address Zip,Vicariate Name,Mailing Address City2,mbfc__Church_Type__c,Parent_Archdpdx_Migration_Id__c,RecordTypeId,Job_Id__c,ParentId,Religious_Type__c,Archdpdx_Migration_Id__c
248,74,Religious,"Society of Mary, Washington, DC (SM)",NaN,No,NaN,"Society of Mary, USA Province",815 Varnum St NE,Washington,DC,...,NaN,NaN,NaN,NaN,submariaenomine,012Dx0000009TK0IAM,49,001Dx00001FZmaaIAD,Local Community,RelCommunities_74
242,68,Religious,"Paulist Fathers, New York, NY (CSP)",NaN,No,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,societassacerdotummissionariorumasanctop,012Dx0000009TK0IAM,49,001Dx00001FZmaUIAT,Local Community,RelCommunities_68
245,71,Religious,"Oblates of the Virgin Mary, Boston, MA (OMV)",NaN,No,NaN,2 Ipswich Street,NaN,Boston,MA,...,NaN,NaN,NaN,NaN,oblatesofthevirginmary,012Dx0000009TK0IAM,49,001Dx00001FZmaXIAT,Local Community,RelCommunities_71
235,61,Religious,Society of Christ Fathers Province in the Unit...,NaN,No,NaN,Society of Christ,786 West Sunset Ave,Lombard,IL,...,NaN,NaN,NaN,NaN,towarzystwochrystusowe,012Dx0000009TK0IAM,49,001Dx00001FZmaOIAT,Local Community,RelCommunities_61
204,27,Religious,"Servite Friars, Portland (OSM)",NaN,No,8719 NE Beech St,Monastery of Our Sorrowful Mother,PO Box 20008,Portland,OR,...,NaN,NaN,NaN,NaN,ordoservorumbeataemariaevirginis,012Dx0000009TK0IAM,49,001Dx00001FZmZxIAL,Local Community,RelCommunities_27


In [55]:
acc_religious_staging_2 = acc_religious_staging[[
    'Name',
    'RecordTypeId',
    'Religious_Type__c',
    'BillingStreet',
    'BillingCity',
    'BillingState',
    'BillingPostalCode',
    'BillingCountry',
    'Phone',
    'Fax',
    'mbfc__Email__c',
    'Website',
    'mbfc__Abbreviation__c',
    'mbfc__Religious_Suffix__c',
    'mbfc__Type_Members__c',
    'Description',
    'Job_Id__c',
    'ParentId',
    'Archdpdx_Migration_Id__c'
    ]]

acc_religious_staging_2.sample(5)

,Name,RecordTypeId,Religious_Type__c,BillingStreet,BillingCity,BillingState,BillingPostalCode,BillingCountry,Phone,Fax,mbfc__Email__c,Website,mbfc__Abbreviation__c,mbfc__Religious_Suffix__c,mbfc__Type_Members__c,Description,Job_Id__c,ParentId,Archdpdx_Migration_Id__c
223,"Sisters of Mercy of the Americas, Portland (RSM)",012Dx0000009TK0IAM,Local Community,2010 NE 19th Ave,Portland,OR,97212,NaN,971-544-7251,NaN,srgeorgita@gmail.com,https://www.mercyworld.org/,Sisters of Mercy,RSM,Women,NaN,49,001Dx00001FZmaDIAT,RelCommunities_47
219,"Good Shepherd Sisters, Portland (RGS)",012Dx0000009TK0IAM,Local Community,5912 NE 16th Ave,Portland,OR,97211,NaN,971-235-2419,NaN,NaN,https://sistersofthegoodshepherd.com/,Good Shepherd Sisters,RGS,Women,NaN,49,001Dx00001FZma9IAD,RelCommunities_43
243,"Redemptorists, Rome, Italy (CSsR)",012Dx0000009TK0IAM,Local Community,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Redemptorists,CSsR,Men,NaN,49,001Dx00001FZmaVIAT,RelCommunities_69
232,"Sacred Heart Maronite Monastery, Beaverton (MM...",012Dx0000009TK0IAM,Local Community,PO Box 13723,Portland,OR,97213,NaN,503-690-4425,NaN,maronitemonks@gmail.com,https://www.maronitemonastery.com/,Maronite Monks,MMJMJ,Men,A contemplative order of Maronite Eastern Cath...,49,001Dx00001FZmaMIAT,RelCommunities_56
196,"Félix Rougier House of Studies, Mount Angel (M...",012Dx0000009TK0IAM,Local Community,PO Box 499,Saint Benedict,OR,97373,NaN,503-845-1181,NaN,rougierhos@gmail.com,https://hos.mspsusa.org/home,Missionaries of the Holy Spirit,MSpS,Men,NaN,49,001Dx00001FZmb9IAD,RelCommunities_14


In [56]:
# Final Cleanup

acc_religious_staging_2 = acc_religious_staging_2.fillna('')

In [59]:
# @title Send to CSV
acc_religious_staging_2.to_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/religious_community_staging.csv', encoding='utf-8-sig')

In [60]:
# @title Upsert to Salesforce
bulk_data = []
for row in acc_religious_staging_2.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    religious_community_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_community_upsert = pd.DataFrame(religious_community_upsert)

df_rel_community_upsert

,success,created,id,errors
0,True,False,001Dx00001FaTiFIAV,[]
1,True,False,001Dx00001FaTiGIAV,[]
2,True,False,001Dx00001FaTiHIAV,[]
3,True,False,001Dx00001FaTiIIAV,[]
4,True,False,001Dx00001FaTiJIAV,[]
...,...,...,...,...
65,True,False,001Dx00001FaTjHIAV,[]
66,True,False,001Dx00001FaTjIIAV,[]
67,True,False,001Dx00001FaTjJIAV,[]
68,True,False,001Dx00001FaTjKIAV,[]


### E) Religious Superiors


In [61]:
acc_rel_superiors = acc_religious_2[[
    'Name',
    'Major Superior Name',
    'Major Superior Phone',
    'Major Superior Email',
    'Archdpdx_Migration_Id__c']]


acc_rel_superiors['AccountId'] = acc_rel_superiors.Archdpdx_Migration_Id__c.map(religious_order_mapping)

# acc_rel_superiors.sample(5)

/var/folders/_g/p51k4k2d5fg45slnqjhgd3sc0000gn/T/ipykernel_47420/2858631496.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_rel_superiors['AccountId'] = acc_rel_superiors.Archdpdx_Migration_Id__c.map(religious_order_mapping)


In [62]:
# @title Parse Complex Names
def parse_names(df, column_name):
    # Convert all non-string entries to strings (handling NaN and other data types)
    df[column_name] = df[column_name].fillna('').apply(str)

    # Create a new DataFrame to store the name parts
    name_parts = pd.DataFrame()

    # Parse each name in the column
    name_parts['First Name'] = df[column_name].apply(lambda x: HumanName(x).first if x.strip() != '' else '')
    name_parts['Last Name'] = df[column_name].apply(lambda x: HumanName(x).last if x.strip() != '' else '')
    name_parts['Middle Name'] = df[column_name].apply(lambda x: HumanName(x).middle if x.strip() != '' else '')
    name_parts['Title'] = df[column_name].apply(lambda x: HumanName(x).title if x.strip() != '' else '')
    name_parts['Suffix'] = df[column_name].apply(lambda x: HumanName(x).suffix if x.strip() != '' else '')
    name_parts['Nickname'] = df[column_name].apply(lambda x: HumanName(x).nickname if x.strip() != '' else '')

    # Combine the original DataFrame with the name parts DataFrame
    result_df = pd.concat([df, name_parts], axis=1)
    return result_df



In [63]:
!pip install nameparser
from nameparser import HumanName
from nameparser.config import CONSTANTS

# Add dataset-specific Titles and Suffix constants for parsing
CONSTANTS.titles.add('Very', 'Rev.', 'Very Rev.', 'Sr.')
CONSTANTS.suffix_acronyms.add('FRS', 'OMI', 'OSA', 'OCD', 'OP', 'OC', 'FSE', 'OMV', 'SDB', 'SM', 'SFX', 'SP', 'OP', 'O.S.M', 'SNJM', 'OSF', 'HMRF', 'DD', 'CSJP', 'SDD', 'BVM', 'BVM - President' )


  Using cached nameparser-1.1.3-py2.py3-none-any.whl.metadata (6.1 kB)
Using cached nameparser-1.1.3-py2.py3-none-any.whl (24 kB)


SetManager({'chi', 'cpre', 'rpa', 'ncto', 'cchp', 'cbsp', 'aas', 'nbcch-ps', 'mai', 'cplp', 'ipep', 'sphr', 'qsp', 'sscp', 'cce', 'thd', 'clsd', 'cprc', 'amieee', 'fasla', 'cfm', 'od', 'ndtr', 'cro', 'lmsw', 'cera', 'litk', 'csbe', 'cip', 'faspen', 'afc', 'crtt', 'dpm', 'cgm', 'rvm', 'rmsks', 'ccc', 'ocm', 'sccp', 'mnae', 'rdh', 'ccp', 'cfr', 'dabfm', 'mpa', 'psm i', 'sdb', 'cem', 'awb', 'pmi-acp', 'bcl', 'capm', 'dep', 'cpe', 'ei', 'ph', 'ncarb', 'se', 'drmp', 'dsm', 'cpacc', 'cga', 'cpm', 'facem', 'mlse', 'csc', 'abr', 'fac-c', 'thm', 'ccpr', 'iccm-d', 'siie', 'sm', 'faafp', 'fic', 'nremr', 'cysa', 'ciro', 'cgfo', 'nbcdch', 'gmr', 'dds', 'mlt', 'cpbe', 'cgc', 'gm', 'cyds', 'sra', 'cela', 'litp', 'fec', 'kcb', 'mp', 'fca', 'chdm', 'usar', 'dpt', 'aphr', 'djur', 'cissp', 'mieee', 'scmp', 'rrt', 'cwna', 'qpm', 'diplom', 'cmc', 'gsmieee', 'cpsc', 'afm', 'git', 'cscu', 'cams', 'dbe', 'ot', 'facc', 'qsd', 'ctfa', 'sfp', 'pfmp', 'dfc', 'cpt', 'bca', 'cfp', 'nrp', 'nicet i', 'ceh', 'ccp-c', 

In [64]:
# Parse Complex Names
acc_rel_superiors_parsed = parse_names(acc_rel_superiors, 'Major Superior Name')

/var/folders/_g/p51k4k2d5fg45slnqjhgd3sc0000gn/T/ipykernel_47420/520038690.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].fillna('').apply(str)


In [65]:
# @title Final cleanup

acc_rel_superiors_staging = acc_rel_superiors_parsed.fillna('')

acc_rel_superiors_staging['Archdpdx_Migration_Id__c'] = acc_rel_superiors_staging['Major Superior Name'].apply(lambda x: x.replace(' ','').lower())

# Rename columns
acc_rel_superiors_staging = acc_rel_superiors_staging.rename(columns={
    'Major Superior Phone': 'Phone',
    'Major Superior Email': 'Email',
    'Title': 'Salutation',
    'First Name': 'FirstName',
    'Middle Name': 'MiddleName',
    'Last Name': 'LastName'
})

# Add job id
acc_rel_superiors_staging['Archdpdx_Job_Id__c'] = curr_job_id

# Drop columns
acc_rel_superiors_staging = acc_rel_superiors_staging.drop(columns=['Name', 'Major Superior Name', 'Nickname'])

# Drop empty rows
acc_rel_superiors_staging = acc_rel_superiors_staging[acc_rel_superiors_staging['LastName'].str.strip() != '']

acc_rel_superiors_staging.sample(10)

,Phone,Email,Archdpdx_Migration_Id__c,AccountId,FirstName,LastName,MiddleName,Salutation,Suffix,Archdpdx_Job_Id__c
211,,,"sr.annemariewarren,osf",001Dx00001FZma2IAD,Anne,Warren,Marie,Sr.,OSF,49
204,773-533-0360 x218,,"veryrev.eugenem.smith,o.s.m",001Dx00001FZmZxIAL,O.S.M,Very Rev. Eugene M. Smith,,,,49
224,,,"sisterbarbaraschamber,sp,provinical",001Dx00001FZmaEIAT,Barbara,Schamber,,Sister,"SP, Provinical",49
236,,,fr.gebhardpaulm.sigl,001Dx00001FZmaPIAT,Gebhard,Sigl,Paul M.,Fr.,,49
248,,,"fr.johnlarsen,sm",001Dx00001FZmaaIAD,John,Larsen,,Fr.,SM,49
191,610-502-1732,ajregionalusa@gmail.com,reverendaugustineidra,001Dx00001FZmZmIAL,Augustine,Idra,,Reverend,,49
232,818-626-9193,info@eparchy.org,"bishopa.eilaszaidan,ddmlm,eparchyofourladyofle...",001Dx00001FZmaMIAT,DD,Bishop A. Eilas Zaidan,MLM,,Eparchy of Our Lady of Lebanon of Los Angeles,49
231,610-626-1400,americanprovince@shcj.org,sr.carrolljuliano,001Dx00001FZmaLIAT,Carroll,Juliano,,Sr.,,49
206,,,"sr.elisegarcía,opchapterprioress",001Dx00001FZmZzIAL,OP,Sr. Elise García,Chapter Prioress,,,49
245,617-536-4141,office@omvusa.org,"fr.jimwalther,omv",001Dx00001FZmaXIAT,Jim,Walther,,Fr.,OMV,49


In [66]:
# @title Send to CSV
acc_rel_superiors_staging.to_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/religious_superiors_staging.csv', encoding='utf-8-sig')

In [67]:
# @title Upsert to Salesforce

def find_existing_contact(sf, first_name, last_name):
    query = f"SELECT Id, Archdpdx_Migration_Id__c FROM Contact WHERE FirstName = '{first_name}' AND LastName = '{last_name}'"
    result = sf.query(query)
    return result['records']



bulk_data = []
for row in acc_rel_superiors_staging.itertuples(index=False):
    d = row._asdict()
    existing_contacts = find_existing_contact(sf, d['FirstName'], d['LastName'])
    if existing_contacts:
        # Update existing contact with external ID
        d['Id'] = existing_contacts[0]['Id']
        bulk_data.append(d)
    else:
        bulk_data.append(d)


if run_upserts == 'True':
    religious_superior_upsert = sf.bulk.Contact.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_superior_upsert = pd.DataFrame(religious_superior_upsert)

df_rel_superior_upsert

,success,created,id,errors
0,False,False,None,"[{'statusCode': 'DUPLICATE_VALUE', 'message': ..."
1,True,False,003Dx00000m0jAWIAY,[]
2,True,False,003Dx00000m0jAXIAY,[]
3,True,False,003Dx00000m0jAYIAY,[]
4,False,True,None,"[{'statusCode': 'INVALID_EMAIL_ADDRESS', 'mess..."
5,True,False,003Dx00000m0jAZIAY,[]
6,True,False,003Dx00000m0jAaIAI,[]
7,False,False,None,"[{'statusCode': 'DUPLICATE_VALUE', 'message': ..."
8,True,False,003Dx00000m0jAbIAI,[]
9,True,False,003Dx00000m0jAcIAI,[]


In [68]:
# @title Update Religious Communities with Rel. Superior

# TBD: It would take much less time to simply do this post-migration manually.

# CONTACTS


## Extract


In [71]:
df_contacts = (pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')
               .set_index('Record Number', verify_integrity=True)
               .drop(index='recNum') # Drops the extra row that replicates the labels
               .rename(columns=lambda x: x.replace(' ', '_')) # Remove whitespace in column names
)

df_contacts.sample(10)


,Common_Name,Sort_Name,Type(s),Clergy_Status,Religious_Status,Login_ID,Password,Password_Must_be_Changed,Access_Permission,Spouse,...,CARA_Ethnicity,Seminarian_Status,Other_Diaconal_Ministry,Spiritual_Director_Authorized,Link_to_Religious_Community,Place_of_Work,Volunteer_Place,Type_of_Work,Work_Load,Work_Title
Record Number,,,,,,,,,,,,,,,,,,,,,
120,Mr. Daniel Medina-Ochoa,medinaochoa daniel,Archive,NaN,NaN,NaN,NaN,NaN,NaN,0,...,Caucasian/white,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1952,Ms. Peg Ries,ries peg,Staff,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1186,"Rev. Frederick Lucci, OP",lucci frederick,"Priest,Religious",Transferred Out,Transferred Out,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN,NaN,NaN
638,Rev. Stephen Fister,fister stephen,Priest,Senior Status,NaN,ffister,bc7030a6b916b8f84f8109ca10bfb209ead1ce9f9ca949...,No,NaN,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2166,Ms. Maria Wanner,wanner maria,Staff,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2792,"Br. Simon Hepner, OSB",hepner simon,Religious,NaN,Active,shepner,2c4cb75d9ec4a252bb5b3494f0e56ee005c3fd2d91da6a...,Yes,NaN,0,...,NaN,NaN,NaN,NaN,4,Mount Angel Abbey,NaN,Library,Full Time,Library Staff
47,Mrs. Lois Sorensen,sorensen lois elouise,Wife,NaN,NaN,NaN,NaN,NaN,NaN,190,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2536,"Sr. Mary Kim Nhung Hoang, MTG",hoang mary kim nhung,Religious,NaN,Active,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,29,NaN,NaN,NaN,NaN,NaN
1470,Mr. Mike Kerrigan,kerrigan mike,Staff,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


#### Get Photos


In [72]:
import os
import pandas as pd

# def list_jpeg_files(directory):
#     data = []
#     for filename in os.listdir(directory):
#         if filename.endswith(".jpeg") or filename.endswith(".jpg"):  # Checking for jpeg files
#             full_path = os.path.join(directory, filename)
#             data.append({'Filename': filename, 'Full Path': full_path})
#     return pd.DataFrame(data)

# # Specify your directory
# directory = '/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/sql_backup/archdpdx.info backups/public_html/people/graphics/portraits/large'
# jpeg_files_df = list_jpeg_files(directory)


In [73]:
# # Query for the Library
# library_query = "SELECT Id, Name FROM ContentWorkspace WHERE Name = 'ADPDX Person Profile Photos'"
# library_result = sf.query(library_query)

# # Check if the library exists and get its ID
# if library_result['records']:
#     library_id = library_result['records'][0]['Id']
#     print(f"Library ID: {library_id}")

#     # Query for the Folder within the Library
#     folder_query = f"SELECT Id, Name FROM ContentFolder WHERE ParentContentFolderId = '{library_id}'"
#     folder_result = sf.query(folder_query)

#     # Check if the folder exists and get its ID
#     if folder_result['records']:
#         folder_id = folder_result['records'][0]['Id']
#         print(f"Folder ID: {folder_id}")
#     else:
#         print("Folder 'Large JPEGs' not found in the library.")
# else:
#     print("Library 'ADPDX Person Profile Photos' not found.")

## Analysis

Here we check the various columns and their types, count where values exist, count of unique values, sample data, etc.

DF shape:

- 142 columns
- 3017 rows


In [75]:
# Check the original shape of the imported CSV
print(f"Shape of original data set: {df_contacts.shape}")

# export to csv a list of the contact fields with count, unique, top, freq
contacts_describe = df_contacts.describe(include='all').transpose()
contacts_describe.to_csv(f'/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/analysis/contacts_describe.csv')

df_contacts.describe(include='all').transpose()  #initial analysis of the Contacts table

Shape of original data set: (3016, 141)


,count,unique,top,freq
Common_Name,3016,3011,Ms. Leslie Jones,2
Sort_Name,3016,3009,nguyen anthony,3
Type(s),3016,29,Staff,1139
Clergy_Status,1138,8,Transferred Out,462
Religious_Status,902,4,Active,456
...,...,...,...,...
Place_of_Work,269,133,Mount Angel Abbey,37
Volunteer_Place,54,47,Mary’s Woods,4
Type_of_Work,276,117,Pastoral Ministry,30
Work_Load,262,2,Full Time,230


In [76]:
unique_languages = df_contacts['Languages'].unique()
unique_languages

array([nan, 'English,Spanish', 'Igbo', 'English, Spanish',
       'Spanish, Mayaqeqchi', 'Spanish (Mass only)',
       'Latin Mass and written translation. Read French, Italian, Spanish.',
       'Spanish', 'Hindi, Konkani, Tamil',
       'French (fluent), Spanish (beginner), Latin (beginner)',
       'German, Spanish, Italian, French', 'Kiswahili, Kichagga',
       'Spanish (English is second language)',
       'German, Spanish, Italian, Latin Mass',
       'English, Spanish, Italian', 'Spanish, Italian', 'English',
       'Bicolango, Tagalog, Spanish', 'Spanish, Italian, Latin Mass',
       'Italian', 'Tagalog, English, Spanish',
       'French, Italian, Aramaic (modern), Spanish', 'Vietnamese',
       'German, Spanish', 'English,Spanish,Italian',
       'Conversant in Italian and Spanish, some facility with Latin and German',
       'English, Spanish, Latin Mass', 'Italian, Spanish',
       'Konkani, Hindi, Marathi, Spanish',
       'Tagalog, Bicol, Spanish (Mass only)', 'Spanish, E

In [77]:
import re
import numpy as np


def deduplicate_languages(list_languages):
    # Define a regular expression pattern to match periods and punctuation
    punctuation_pattern = r'[.,!?;:"]'

    # Flatten the array and filter out NaN values
    flattened_languages = [re.sub(punctuation_pattern, '', lang) for sublist in list_languages if pd.notna(sublist) for lang in sublist.split(',')]

    # Deduplicate the list of languages
    unique_languages = list(set(flattened_languages))

    return unique_languages


# Example usage:
unique_languages = deduplicate_languages(unique_languages)
print(unique_languages)


['', ' Spanish (small bits)', ' German (small bits)', 'Crijolle', ' Hebrew (reading)', 'Igbo', ' Little Spanish', ' Swahili', 'Italian', 'Korean', ' Englsih', ' but can do rituals)', ' Arabic', ' Maya Q’eqchi’', ' Hindi', ' Konkani', ' Telugu', ' Latin (beginner)', ' Aramaic (modern)', 'Vietnamese (Mass only)', ' Spanish Mass', ' English', 'Afrikaans', ' Kiswahili', 'Portuguese', 'German', ' Marathi', 'Spanish (English is second language)', 'Kannada', 'English', ' Latin Mass', 'Latin Mass and written translation Read French', 'Polish', 'Chagga', 'Tamil', ' Mayaqeqchi', ' Konkawin', ' Bicol', 'Hindi', 'Chuukese', 'Spanish (Mass only)', 'Kisii', ' Greek', ' Latin', ' French', ' Vietnamese', 'Vietnamese', 'Tamil (Indian)', 'Spanish', 'Konkani', 'French (small bits)', ' a little Spanish(not conversational', 'Latin Mass', ' Chamorro', ' Tamil', ' Kichagga', ' some facility with Latin and German', 'Latin', ' Ukrainian', ' Vietnamese (semi-fluent)', 'French', ' Portuguese', ' Spanish (Mass on

## Transform


In [78]:
# init list of columns NOT to be loaded as Contact attributes
misc_columns_to_drop = [
    'Password',
    'Password_Must_be_Changed',
    'Common_Name',
    'Sort_Name',
    'Private_Address_Province'
]

affiliation_columns = [
    'Seminarian_Student_Debt',
    'Seminarian_Medical_Benefits',
    'Baptism_Date',
    'Place_of_Baptism',
    'Confirmation_Date',
    'Place_of_Confirmation',
    'Received_Date',
    'Parish_of_Record',
    'Marriage_Date',
    'Place_of_Marriage',
    'Date_of_First_Vows',
    'Date_of_Final_Vows',
    'Accepted_to_Formation_Date',
    'Reader_Date',
    'Acolyte_Date',
    'Candidacy_Date',
    'Formation_Withdrawn_Date',
    'Formation_Deferred_Date',
    'Formation_Terminated_Date',
    'Terminate_or_Defer_Note',
    'Bachelor_Degree_Year',
    'Bachelor_Degree_Type',
    'Bachelor_Degree_Institution',
    'Graduate_1_Degree_Institution',
    'Graduate_1_Degree_Type',
    'Graduate_1_Degree_Year',
    'Graduate_2_Degree_Institution',
    'Graduate_2_Degree_Type',
    'Graduate_2_Degree_Year',
    'Graduate_3_Degree_Institution',
    'Graduate_3_Degree_Type',
    'Graduate_3_Degree_Year',
    'Graduate_4_Degree_Institution',
    'Graduate_4_Degree_Type',
    'Graduate_4_Degree_Year',
    'CARA_Highest_Ed_Level',
    'Diaconal_Ordination_Date',
    'Diaconal_Ordination_Place',
    'Diaconal_Ordination_Prelate',
    'Presbyteral_Ordination_Date',
    'Presbyteral_Ordination_Place',
    'Presbyteral_Ordination_Prelate',
    'Episcopal_Ordination_Date',
    'Episcopal_Ordination_Place',
    'Episcopal_Ordination_Prelate',
    'Ordination_Diocese',
    'Incardinated_From_Date',
    'Incardinated_From_Diocese',
    'Incardinated_Now',
    'Serving_Now',
    'Excardinated_To_Diocese',
    'Excardinated_To_Date',
    'Letter_of_Good_Standing_Date',
    'Religious_In_Archdiocese_Date',
    'Faculties',
    'Faculties_Granted_Date',
    'Faculties_Restricted_Date',
    'Faculties_Withdrawn_Date',
    'Last_Retreat_Date',
    'Last_Educ_Requirement_Date',
    'Policy_Manual_Acknowledgement_Date',
    'Harassment_Prevention_Course_Date',
    'Standards_of_Conduct_Date',
    'Last_Background_Check_Date',
    'Last_Child_Protection_Training_Date',
    'Out_of_Diocese_Date',
    'Senior_Status_Date',
    'Laicized_Date',
    'Coverage_Availability',
    'Advanced_Directive_Date',
    'End_of_Life_Plan_Date',
    'Will_Date',
    'Will_Note',
    'CIC_489_File',
    'Registered_Parish',
    'CARA_Ethnicity',
    'Seminarian_Status',
    'Other_Diaconal_Ministry',
    'Spiritual_Director_Authorized',
    'Link_to_Religious_Community',
    'Place_of_Work',
    'Volunteer_Place',
    'Type_of_Work',
    'Work_Load',
    'Work_Title'
]

In [79]:
# These fields need to be KEPT but while building the SF upsert flow these are dropped temporarily until mapping logic is included.
# TODO

fields_not_yet_mapped = [
    'Spouse',
    'Father_Full_Name',
    'Mother_Full_Maiden_Name',
    'Mailing_Address_2',
    'Mailing_Address_Province',
    'Private_Address_2',
    'Nickname',
    'Preferred_Address',
    'Private_Address__Street__s',
    'Private_Address__City__s',
    'Private_Address__StateCode__s',
    'Private_Address__PostalCode__s',
    'Private_Address__CountryCode__s',
    'Social_Security_Account_Number__c',  # The data is encrypted
    'Languages',  # Picklist is restricted, in MFC package. Needs unrestricting before I can migrate data.
    'Preferred_Email',
    'Preferred_Phone'

]

In [80]:
# UDF to combine multiple Mailing Street Address lines into one

def combine_addresses(row, *columns):
    address_parts = []
    for col in columns:
        address_parts.append(row[col])
    return 'CHAR(10)'.join(address_parts)


In [81]:
df_contact_staging = (df_contacts
                      .drop(columns='Salutation')
                      .rename(columns={
                          'Clergy_Status' : 'ADPDX_Clergy_Status__c',
                          'Religious_Status' : 'ADPDX_Religious_Status__c',
                          'Login_ID' : 'ADPDX_Login_ID__c',
                          'Access_Permission': 'ADPDX_Access_Permission__c',
                          'Title': 'Salutation',
                          'Christian_Name': 'FirstName',
                          'Middle_Name(s)': 'MiddleName',
                          'Surname': 'LastName',
                          'Suffix': 'Suffix',
                          #Mailing_Address & Mailing_Address_2
                          'Mailing_Address' : 'MailingStreet',
                          'Mailing_Address_City': 'MailingCity',
                          'Mailing_Address_State': 'MailingState',
                          #'Mailing_Address_Province': 'MailingProvince'
                          'Mailing_Address_Postal_Code': 'MailingPostalCode',
                          'Mailing_Address_Country': 'MailingCountry',
                          'Private_Address': 'Private_Address__Street__s',
                          'Private Address 2': 'Private_Address__Street__s',
                          'Private_Address_City': 'Private_Address__City__s',
                          'Private_Address_State': 'Private_Address__StateCode__s',
                          'Private_Address_Postal_Code': 'Private_Address__PostalCode__s',
                          'Private_Address_Country': 'Private_Address__CountryCode__s',
                          # 'Preferred_Address'
                          'Work_Phone': 'npe01__WorkPhone__c',
                          'Home_Phone': 'HomePhone',
                          'Cell_Phone': 'MobilePhone',
                        #   'Preferred_Phone': 'npe01__PreferredPhone__c',
                          # IF Preferred phone contains, 'do not publish'
                          'Work_Email' : 'npe01__WorkEmail__c',
                          'Archdiocesan_Email': 'npe01__AlternateEmail__c',
                          'Home_Email': 'npe01__HomeEmail__c',
                        #   'Preferred_Email': 'npe01__Preferred_Email__c',
                          # IF Preferred email contains 'do not publish''
                          'Directory_Include': 'Directory_Include__c',
                          'Directory_Include_Middle_Name': 'Directory_Include_Middle_Name__c',
                          'Directory_Include_Suffix': 'Directory_Include_Suffix__c',
                          'Suppress_From_Reports': 'Suppress_From_Reports__c',
                          'Send_Group_Mail_and_Email': 'Send_Group_Mail_and_Email__c',
                          'Birth_Date': 'Birthdate',
                          'Place_of_Birth': 'mbfc__Place_of_Birth__c',
                          'Foreign_Born': 'Foreign_Born__c',
                          'Foreign_Citizenship': 'Foreign_Citizenship__c',
                          'Immigration_Status': 'Immigration_Status__c',
                          'Passport/Visa_Expiration_Date': 'Passport_Visa_Expiration_Date__c',
                          'Social_Security_Account_Number': 'Social_Security_Account_Number__c',
                          'Deceased_Date': 'mbfc__Date_of_Death__c',
                        #   'Languages': 'Languages__c'
                          })
                      .assign(Bi_Ritual__c=lambda x: x['Type(s)'].str.contains('Biritual'))
                      .assign(Non_Latin_Rite__c=lambda x: x['Type(s)'].str.contains('Non-Latin Rite'))
                      .assign(Archdpdx_Migration_Id__c=lambda x: x.index)
                    #   .assign(Mailing_Address=lambda row: combine_addresses(row, 'Mailing_Address', 'Mailing_Address_2'), axis=1)
                      .drop(columns=misc_columns_to_drop)
                      .drop(columns=affiliation_columns)
                      .drop(columns=fields_not_yet_mapped)

        )


df_contact_staging.sample(10)

,Type(s),ADPDX_Clergy_Status__c,ADPDX_Religious_Status__c,ADPDX_Login_ID__c,ADPDX_Access_Permission__c,Salutation,FirstName,MiddleName,LastName,Suffix,...,Birthdate,mbfc__Place_of_Birth__c,Foreign_Born__c,Foreign_Citizenship__c,Immigration_Status__c,Passport_Visa_Expiration_Date__c,mbfc__Date_of_Death__c,Bi_Ritual__c,Non_Latin_Rite__c,Archdpdx_Migration_Id__c
Record Number,,,,,,,,,,,,,,,,,,,,,
2829,Staff,NaN,NaN,dramsden,school-*,Mrs.,Denise,NaN,Ramsden,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,2829
72,Permanent Deacon,Deceased,NaN,NaN,NaN,Deacon,David,Francis,Partlow,NaN,...,1934-05-07,Dayton OH,No,NaN,NaN,NaN,2013-06-01,False,False,72
1824,Staff,NaN,NaN,NaN,NaN,Ms.,Sue,NaN,Unger,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1824
1525,"Priest,Religious",Active,Active,wameche,NaN,Rev.,William,A.,Ameche,NaN,...,1947-01-10,"Los Angeles, CA",No,NaN,NaN,NaN,NaN,False,False,1525
3035,Religious,NaN,Active,NaN,NaN,Sr.,Trúc Grace,NaN,Nguyen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,3035
1618,Staff,NaN,NaN,NaN,NaN,Mr.,Wells,NaN,O’Byrne,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1618
1851,Staff,NaN,NaN,NaN,NaN,Mr.,Brandon,NaN,Swaney,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1851
1237,"Priest,Religious",Transferred Out,Transferred Out,NaN,NaN,Rev.,Joseph,NaN,Moyer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1237
3155,Staff,NaN,NaN,NaN,NaN,Ms.,Anna,NaN,Lake,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,3155


### Private Address Handling


In [82]:
# df_contact_staging.loc[:,'Private_Address__Street__s':'Private_Address__CountryCode__s'][~df_contact_staging['Private_Address__StateCode__s'].isna()]

In [83]:
# df_contact_staging['Private_Address__CountryCode__s'] = df_contact_staging.apply(lambda row: 'United States' if pd.notnull(row['Private_Address__StateCode__s']) and pd.isnull(row['Private_Address__CountryCode__s']) else row['Private_Address__CountryCode__s'], axis=1)

### Handle Boolean Fields


In [84]:
boolean_columns_to_convert = ['Foreign_Born__c', 'Directory_Include__c', 'Directory_Include_Middle_Name__c', 'Directory_Include_Suffix__c',
       'Suppress_From_Reports__c', 'Send_Group_Mail_and_Email__c', ]

df_contact_staging[boolean_columns_to_convert] = df_contact_staging[boolean_columns_to_convert].replace({'Yes': True, 'No': False})


In [85]:
df_contact_staging[boolean_columns_to_convert] = df_contact_staging[boolean_columns_to_convert].fillna(False)

df_contact_staging[boolean_columns_to_convert].sample(5)

,Foreign_Born__c,Directory_Include__c,Directory_Include_Middle_Name__c,Directory_Include_Suffix__c,Suppress_From_Reports__c,Send_Group_Mail_and_Email__c
Record Number,,,,,,
849,False,False,False,False,False,False
2941,False,False,False,False,False,True
2699,False,False,False,False,False,True
826,False,True,False,False,False,True
2567,False,True,False,False,False,True


### Set Contact Record Type


In [86]:
# Set Record Type

# Go down row by row and check the 'Type(s)' columns, check for certain words that are keys in a dictionary, and
# the that row's 'Type(s)' field contains a string that is in the a key in a dictionary the update another columns
# called 'ContactRecordType' with the paired value.

contact_type_map = {
    'Bishop': 'Priest',
    'Diaconate': 'Lay_Person',
    'Permanent Deacon': 'Permanent_Deacon',
    'Priest': 'Priest',
    'Staff': 'Lay_Person',
    'Seminarian': 'Lay_Person',
    'Wife': 'Lay_Person',
    'Religious': 'Religious',
    'Seminary Applicant': 'Lay_Person',
    'Transitional Deacon': 'Priest',
    'Archive': 'Lay_Person'
}

def update_contact_record_type(row):
    for key, value in contact_type_map.items():
        if key in row['Type(s)']:
            return value
    return None

df_contact_staging['ContactRecordType'] = df_contact_staging.apply(update_contact_record_type, axis=1)

In [87]:
# Map in the RecordTypeIDs
df_contact_staging['RecordTypeID'] = df_contact_staging['ContactRecordType'].map(record_types_mapping)

In [88]:
# Check for any Contacts who are missing a RecordTypeId
df_contact_staging[df_contact_staging['RecordTypeID'].isna()]

,Type(s),ADPDX_Clergy_Status__c,ADPDX_Religious_Status__c,ADPDX_Login_ID__c,ADPDX_Access_Permission__c,Salutation,FirstName,MiddleName,LastName,Suffix,...,Foreign_Born__c,Foreign_Citizenship__c,Immigration_Status__c,Passport_Visa_Expiration_Date__c,mbfc__Date_of_Death__c,Bi_Ritual__c,Non_Latin_Rite__c,Archdpdx_Migration_Id__c,ContactRecordType,RecordTypeID
Record Number,,,,,,,,,,,,,,,,,,,,,


### Email Validation


In [89]:
# !pip install email_validator
from email_validator import validate_email, EmailNotValidError

# function that validates an email and if it is invalid it returns nothing
def validate_email_address(email):
    if isinstance(email, float):
        return None
    try:
        v = validate_email(email)
        return v.email
    except EmailNotValidError:
        return None


In [90]:
# init a list of Email columns
# email_columns = ['npe01__HomeEmail__c', 'npe01__WorkEmail__c', 'npe01__AlternateEmail__c']

# df_contact_staging[email_columns] = df_contact_staging[email_columns].applymap(validate_email_address)

### Final Dataframe Cleanup


In [91]:
# drop columns that are no longer needed
del df_contact_staging['Type(s)']
del df_contact_staging['ContactRecordType']

In [92]:
# clean up all NaN values

df_contact_staging = df_contact_staging.fillna('')

## Load


In [93]:
df_contact_staging['Archdpdx_Job_Id__c'] = curr_job_id

In [95]:
# generate CSV for manual loading
df_contact_staging.to_csv(f'/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/df_contacts_staging.csv', encoding='utf-8-sig')
df_contact_staging.to_csv('contacts_staging.csv', encoding='utf-8-sig')


In [96]:
# upsert Contact records into SF using Bulk api

from simple_salesforce.exceptions import SalesforceMalformedRequest

bulk_data = []
for row in df_contact_staging.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

try:
    # Attempt to upsert Contact records into SF using Bulk API
    contact_upsert = sf.bulk.Contact.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=True)
    contact_upsert_results = pd.DataFrame(contact_upsert)
except SalesforceMalformedRequest as e:
    # If a SalesforceMalformedRequest error occurs, print the error message and response content
    print(f"SalesforceMalformedRequest error: {e}")
    print(f"Response content: {e.content}")

SalesforceMalformedRequest error: Malformed request https://adpdx--devpro.sandbox.my.salesforce.com/services/async/57.0/job/750Dx000007zgc7IAA/batch/751Dx000009HBkFIAW/result. Response content: {'exceptionCode': 'InvalidBatch', 'exceptionMessage': 'Records not processed'}
Response content: {'exceptionCode': 'InvalidBatch', 'exceptionMessage': 'Records not processed'}


In [98]:
# Print upsert results to local file

# keys = contact_upsert[0].keys()

# with open('contact_results', 'w', newline='') as csv_file:
#     writer = csv.DictWriter(csv_file, keys)
#     writer.writeheader()
#     writer.writerows(contact_upsert)


# CONTACT > REGISTER ENTRIES


In [100]:
import pandas as pd

# Load CSV
df = (pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')
               .rename(columns=lambda x: x.replace(' ', '_')) # Remove whitespace in column names
               .drop(index=0) # Drops the extra row that replicates the labels
)

df

,Record_Number,Common_Name,Sort_Name,Type(s),Clergy_Status,Religious_Status,Login_ID,Password,Password_Must_be_Changed,Access_Permission,...,CARA_Ethnicity,Seminarian_Status,Other_Diaconal_Ministry,Spiritual_Director_Authorized,Link_to_Religious_Community,Place_of_Work,Volunteer_Place,Type_of_Work,Work_Load,Work_Title
1,2766,Rev. Stephen Abaukaka,abaukaka stephen ozovehe,Priest,Transferred Out,NaN,sabukaka,def2a990be60a7998b1ed7c820101f3bd02d33b8992518...,Yes,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2337,Mr. Rogelio Acevedo,acevedo rogelio,Staff,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,3244,Mr. Sean Ackroyd,ackroyd sean,Staff,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4,3295,Ms. Sherril Acton,acton sherril,Staff,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
5,2164,Ms. Barbara Adams,adams barbara,Staff,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,1670,Ms. Jenny Zomerdyk,zomerdyk jenny,Staff,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3013,2755,"Br. Daniel Zorrilla, MSpS",zorrilla daniel,Religious,NaN,Active,dzorrilla,391eedf7c936f63d3d0a7d9ea7e506a84709662fd31ba9...,Yes,NaN,...,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN
3014,1962,Ms. Kim Zuber,zuber kim,Staff,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3015,2202,Ms. Agnes Zueger,zueger agnes,Staff,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [101]:
# @title Parse Sacrament & Notation Types

#TODO: Baptisms should be set to 'Notice of Baptism', not 'Baptism

import pandas as pd

# Define the structure of your column sets with correct attribute names
column_sets = [
    {'date': 'Baptism_Date', 'place': 'Place_of_Baptism', 'notation_type': 'Proof of Baptism'},
    {'date': 'Confirmation_Date', 'place': 'Place_of_Confirmation', 'notation_type': 'Notice of Confirmation'},
    {'date': 'Received_Date', 'place': 'Parish_of_Record', 'notation_type': 'Notation of Profession of Faith'},
    {'date': 'Marriage_Date', 'place': 'Place_of_Marriage', 'notation_type': 'Notice of Matrimony'},
    {'date': 'Diaconal_Ordination_Date', 'place': 'Diaconal_Ordination_Place', 'prelate': 'Diaconate_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Diaconate'},
    {'date': 'Presbyteral_Ordination_Date', 'place': 'Presbyteral_Ordination_Place', 'prelate': 'Presbyteral_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Presbyteral'},
    {'date': 'Episcopal_Ordination_Date', 'place': 'Episcopal_Ordination_Place', 'prelate': 'Episcopal_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Episcopal'}
]

# New DataFrame for entries
register_entries = pd.DataFrame(columns=['RecordNumber', 'mbfc__Register_Entry_Type__c', 'mbfc__Type__c', 'mbfc__Notation_Type__c', 'mbfc__Ordination_Type__c', 'Date', 'Place', 'Prelate'])
new_entries = []  # List to store entries before final concatenation

# Processing rows
for row in df.itertuples():
    for column_set in column_sets:
        date_value = getattr(row, column_set['date'], None)
        if pd.notna(date_value):  # Check if date field is not NaN
            entry = {
                'RecordNumber': getattr(row, 'Record_Number', None),
                'Date': date_value,
                'Place': getattr(row, column_set['place'], None)
            }
            # Add Prelate if applicable
            if 'prelate' in column_set:
                entry['Prelate'] = getattr(row, column_set['prelate'], None)

            # Set 'mbfc__Register_Entry_Type__c', and conditionally add 'mbfc__Type__c' or 'mbfc__Notation_Type__c'
            if 'sacrament_type' in column_set:
                entry['mbfc__Type__c'] = column_set['sacrament_type']
                entry['mbfc__Register_Entry_Type__c'] = 'Sacrament'
            if 'notation_type' in column_set:
                entry['mbfc__Notation_Type__c'] = column_set['notation_type']
                entry['mbfc__Register_Entry_Type__c'] = 'Notation'

            # Handle ordination type specific updates
            if 'ordination_type' in column_set:
                entry['mbfc__Ordination_Type__c'] = column_set['ordination_type']

            new_entries.append(entry)

# Concatenate all new entries to the DataFrame at once
if new_entries:
    register_entries = pd.concat([register_entries, pd.DataFrame(new_entries)], ignore_index=True)

print(f"Total records added: {len(register_entries)}")

# Optionally, save the new DataFrame to a CSV
register_entries.to_csv('Register_Entries.csv', index=False)

# Display the DataFrame
register_entries.sample(10)


Total records added: 1534


,RecordNumber,mbfc__Register_Entry_Type__c,mbfc__Type__c,mbfc__Notation_Type__c,mbfc__Ordination_Type__c,Date,Place,Prelate
1532,787,Notation,NaN,Notice of Holy Orders,Presbyteral,1988-04-16,"Abbey Church, Mount Angel Abbey, Saint Benedic...",Most Rev. William J. Levada
1433,868,Notation,NaN,Notice of Holy Orders,Presbyteral,1982-12-08,"Abbey Church, Mount Angel Abbey, Saint Benedic...",Most Rev. Cornelius Power
967,2068,Notation,NaN,Notice of Confirmation,NaN,1985-08-15,"Jejaf Parish, South Sudan",NaN
1051,1265,Notation,NaN,Notice of Holy Orders,Presbyteral,1973-02-02,"Innsbruck, Austria",Dr. Paulus Rusch
772,165,Notation,NaN,Proof of Baptism,NaN,1953-04-05,"St. Michael’s Church, Palisades Park NJ",NaN
1087,424,Notation,NaN,Notice of Confirmation,NaN,1998-04-11,NaN,NaN
788,252,Notation,NaN,Proof of Baptism,NaN,1966-03-20,"First Christian Church, Santa Ana CA",NaN
1210,2800,Notation,NaN,Proof of Baptism,NaN,1979-07-08,"Christ United Methodist Church, Albuquerque, NM",NaN
1268,2785,Notation,NaN,Proof of Baptism,NaN,1962-04-15,"Assumption of the Blessed Virgin Parish, Kansa...",NaN
624,122,Notation,NaN,Notice of Matrimony,NaN,1964-10-10,NaN,NaN


In [102]:
from nameparser import HumanName
from nameparser.config import CONSTANTS

# Add dataset-specific Titles and Suffix constants for parsing
CONSTANTS.titles.add('Very', 'Rev.', 'Very Rev.', 'Sr.', 'Most Rev.')
CONSTANTS.suffix_acronyms.add('FRS', 'J.C.L.', 'J.C.L., D.D.', 'D.D.', 'OMI', 'OSA', 'OCD', 'OP', 'OC', 'FSE', 'OMV', 'SDB', 'SM', 'SFX', 'SP', 'OP', 'O.S.M', 'SNJM', 'OSF', 'HMRF', 'DD', 'CSJP', 'SDD', 'BVM', 'BVM - President', 'SJ', 'SL', 'IX', 'SSJ', 'J.C.L.', 'J.C.L', 'OFM', 'MSpS', 'Fco.' )


def parse_name(name):
    if pd.isna(name):  # Checks if the name is NaN or None
        return {
            'Salutation': '',
            'FirstName': '',
            'MiddleName': '',
            'LastName': '',
            'Suffix': ''
        }
    else:
        name = HumanName(name)
        return {
            'Salutation': name.title,
            'FirstName': name.first,
            'MiddleName': name.middle,
            'LastName': name.last,
            'Suffix': name.suffix
        }

# Apply the parsing function only where 'Prelate' exists and is not NaN
for entry in new_entries:
    if 'Prelate' in entry and pd.notna(entry['Prelate']):
        parsed_name = parse_name(entry['Prelate'])
        entry.update(parsed_name)

# Ensure the DataFrame creation from new_entries includes checks for existence of keys:
register_entries = pd.DataFrame(new_entries)
if 'Prelate' in register_entries.columns:
    register_entries['Salutation'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['Salutation'] if pd.notna(x) else '')
    register_entries['FirstName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['FirstName'] if pd.notna(x) else '')
    register_entries['MiddleName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['MiddleName'] if pd.notna(x) else '')
    register_entries['LastName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['LastName'] if pd.notna(x) else '')
    register_entries['Suffix'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['Suffix'] if pd.notna(x) else '')


# Display the DataFrame
print(f"Total records added: {len(register_entries)}")
register_entries.sample(10)



Total records added: 1534


,RecordNumber,Date,Place,Prelate,mbfc__Notation_Type__c,mbfc__Register_Entry_Type__c,mbfc__Ordination_Type__c,Salutation,FirstName,MiddleName,LastName,Suffix
1033,809,2005-06-11,"Cathedral of the Immaculate Conception, Portla...",Most Rev. John G. Vlazny,Notice of Holy Orders,Notation,Presbyteral,Most Rev.,John,G.,Vlazny,
53,237,2006-12-09,"Cathedral of the Immaculate Conception, Portla...",None,Notice of Holy Orders,Notation,Diaconate,,,,,
1276,726,1985-04-13,NaN,NaN,Notice of Holy Orders,Notation,Presbyteral,,,,,
908,873,2001-06-09,"Cathedral of the Immaculate Conception, Portla...",Most Rev. John G. Vlazny,Notice of Holy Orders,Notation,Presbyteral,Most Rev.,John,G.,Vlazny,
1371,2832,1994-04-02,"St. Catherine of Siena Parish, Rialto, CA",NaN,Proof of Baptism,Notation,NaN,,,,,
579,715,1970-10-18,"Sublimity, Oregon",Most Rev. Robert J. Dwyer,Notice of Holy Orders,Notation,Presbyteral,Most Rev.,Robert,J.,Dwyer,
1197,101,1999-06-24,"Cathedral of Our Lady of Lourdes, Spokane WA",None,Notice of Holy Orders,Notation,Diaconate,,,,,
1504,3043,2003-05-11,Strara Halic,None,Notice of Holy Orders,Notation,Diaconate,,,,,
394,1582,2013-11-23,"Sacred Heart Cathedral, Eldoret, Kenya",Most. Rev. Cornelius Kipng’eno Arap Korir,Notice of Holy Orders,Notation,Presbyteral,Most. Rev.,Cornelius,Kipng’eno Arap,Korir,
736,1545,2013-02-06,NaN,NaN,Notice of Matrimony,Notation,NaN,,,,,


In [103]:
# @title Query Salesforce for existing contacts and create a dictionary for mapping

from simple_salesforce import Salesforce

query = """
SELECT Id, Archdpdx_Migration_ID__c
FROM Contact
"""
result = sf.query_all(query)
contact_map = {rec['Archdpdx_Migration_ID__c']: rec['Id'] for rec in result['records']}


In [104]:
# Get RecordTypeId for Contact.Priest

priest_contact_recordtype_id = df_sf_recordTypes.loc[
    (df_sf_recordTypes['DeveloperName'] == 'Priest') & (df_sf_recordTypes['SobjectType'] == 'Contact'),
    'Id'
    ].iloc[0]  # Use .iloc[0] to get the first item if you're expecting exactly one match


In [105]:
# @title Query for Contacts by Names and Create New Contacts

from simple_salesforce import SFType, SalesforceResourceNotFound

contact = SFType('Contact', sf.session_id, sf.sf_instance)
for index, row in register_entries.iterrows():
    first_name, last_name = row.get('FirstName'), row.get('LastName')

    if pd.isna(first_name) or pd.isna(last_name) or first_name.strip() == '' or last_name.strip() == '':
        # If either first name or last name is missing or empty, skip this row or handle as needed
        print(f"Skipping row {index} due to missing name information.")
        continue

    try:
        # Search for contact by First and Last Name
        query = f"SELECT Id FROM Contact WHERE FirstName = '{first_name}' AND LastName = '{last_name}'"
        result = sf.query(query)
        if result['totalSize'] > 0:
            contact_id = result['records'][0]['Id']
        else:
            # Create a new contact if no match found
            new_contact = {
                'FirstName': first_name,
                'LastName': last_name,
                'Archdpdx_Job_Id__c': curr_job_id,
                'RecordTypeId': priest_contact_recordtype_id
            }
            create_result = contact.create(new_contact)
            contact_id = create_result['id']

        # Update DataFrame with the Salesforce Contact ID
        register_entries.at[index, 'mbfc__Celebrant__c'] = contact_id

    except SalesforceException as e:
        print(f"Error processing row {index}: {e}")



Skipping row 2 due to missing name information.
Skipping row 3 due to missing name information.
Skipping row 4 due to missing name information.
Skipping row 5 due to missing name information.
Skipping row 6 due to missing name information.
Skipping row 8 due to missing name information.
Skipping row 9 due to missing name information.
Skipping row 10 due to missing name information.
Skipping row 11 due to missing name information.
Skipping row 12 due to missing name information.
Skipping row 13 due to missing name information.
Skipping row 16 due to missing name information.
Skipping row 17 due to missing name information.
Skipping row 18 due to missing name information.
Skipping row 19 due to missing name information.
Skipping row 20 due to missing name information.
Skipping row 21 due to missing name information.
Skipping row 22 due to missing name information.
Skipping row 23 due to missing name information.
Skipping row 24 due to missing name information.
Skipping row 26 due to miss

In [106]:
# @title Map Contact IDs to Register Entries

register_entries_2 = register_entries

register_entries_2['mbfc__Contact__c'] = register_entries['RecordNumber'].map(contact_map)


In [107]:
# @title Append Job_Id__c
register_entries_2['Archdpdx_Job_Id__c'] = curr_job_id

## Generate an External ID


In [108]:
def create_external_id(row):
    record_number = str(row['RecordNumber']).replace(' ', '').replace('-', '')
    entry_type = str(row['mbfc__Register_Entry_Type__c']).replace(' ', '').replace('-', '')

    # Check whether to use Type or Notation Type based on what's available
    if 'mbfc__Type__c' in row and not pd.isna(row['mbfc__Type__c']):
        type_field = str(row['mbfc__Type__c']).replace(' ', '').replace('-', '')
    elif 'mbfc__Notation_Type__c' in row and not pd.isna(row['mbfc__Notation_Type__c']):
        type_field = str(row['mbfc__Notation_Type__c']).replace(' ', '').replace('-', '') + str(row['mbfc__Ordination_Type__c']).replace(' ', '').replace('-', '')
    else:
        type_field = 'Unknown'

    return f"{record_number}_{entry_type}_{type_field}"

In [109]:
# Assuming your DataFrame is named `register_entries`
register_entries_2['Archdpdx_Migration_ID__c'] = register_entries.apply(create_external_id, axis=1)

if register_entries['Archdpdx_Migration_ID__c'].duplicated().any():
    print("Warning: There are duplicate external IDs.")
    # Optionally, show the duplicates
    duplicates = register_entries[register_entries['external_id'].duplicated(keep=False)]
    print(duplicates)
else:
    print("All external IDs are unique.")


All external IDs are unique.


In [110]:
# Drop unnecessary columns:
register_entries_2.drop(['RecordNumber', 'Prelate', 'Salutation', 'FirstName', 'MiddleName', 'LastName', 'Suffix'], axis=1, inplace=True)

In [111]:
register_entries_staging = register_entries_2

In [112]:
# Remove all NaN values:
register_entries_staging.fillna('', inplace=True)

# Rename columns
register_entries_staging = register_entries_staging.rename(columns={
    'Place': 'Location_text__c',
    'Date': 'mbfc__Event_Date__c'
})


In [113]:
# @title Sent to CSV
# register_entries_staging.to_csv('register_entries_staging.csv', encoding='utf-8-sig')

In [114]:
# @title Upsert Register Entry Records

bulk_data = []
for row in register_entries_staging.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

# Keep the batch <100 as I've been getting an exceptionCode: 'InvalidBatch', 'exceptionMessage': 'Records not processed'
reg_entry_upsert = sf.bulk.mbfc__Sacrament__c.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
reg_entry_upsert_results = pd.DataFrame(reg_entry_upsert)

In [115]:
# Print upsert results to local file

keys = reg_entry_upsert[0].keys()

with open('register_entry_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(reg_entry_upsert)

# Users


In [116]:
# df_users = df_contacts[df_contacts['Access Permission'].isna() == False]
# df_users = df_users[['Record Number', 'Common Name', 'Sort Name', 'Type(s)', 'ContactRecordType', 'Login ID', 'Access Permission']]
# df_users.sort_values('Access Permission')
# df_users.to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/users_working.csv')

# AFFILIATIONS


In [117]:
# @title Import Assignments.csv

import pandas as pd

# Define a mapping dictionary of OriginalTable to AccRecordType name
organization_mapping = {
    'Offices': 'Organization',
    'Parishes': 'Church',
    'RelCommunities': 'Religious',
    'Schools': 'School',
    'Vicariates': 'Deanery',
    'NewmanCenters': 'Organization'
}

df_affiliations = (
    pd.read_csv('/Users/matthewmartin/Library/CloudStorage/GoogleDrive-matt@meribahflow.com/Shared drives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Assignments (1).csv')
    .set_index('Record Number', verify_integrity=True)
    .drop(index='recNum', errors='ignore')  # Added errors='ignore' to prevent errors if 'recNum' does not exist
    .drop(columns=['Historic Name'], errors='ignore')  # Added errors='ignore' for the same reason
    .rename(columns=lambda x: x.replace(' ', '_'))  # Remove whitespace in column names
    .assign(Account_Ext_Id=lambda df: df['Organization_Table_Name'].map(organization_mapping).str.capitalize() + '_' + df['Organization_Table_Link'])
    .assign(mbfc__Person__r=lambda df: df['Assigned_Person'].apply(lambda x: {'Adpdx_Migration_Id__c': x}))
    .assign(mbfc__Context__r=lambda df: df['Account_Ext_Id'].apply(lambda x: {'Adpdx_Migration_Id__c': x}))
    .assign(mbfc__Use_Custom_Title__c= True)
    # .assign(Archdpdx_Migration_Id__c= df_affiliations.index)
    .drop(columns=[
        'Assigned_Person'
        ,'Organization_Table_Name'
        ,'Organization_Table_Link'
        ,'Projected_Term_End_Date'
        ,'Term_Number'
        ,'Leave_Type' # Leave out 'Leave_Type' until mapped properly
        ])
    .rename(columns={
        'Duty_Load': 'Duty_Load__c',
        'Start_Date': 'mbfc__Start_Date__c',
        'End_Date': 'mbfc__Completion_Date__c',
        'Assignment_Title': 'mbfc__Custom_Title__c',
        'Archdiocesan_Assignment': 'ADPDX_Archdiocesan_Assignment__c',
    })
    .replace({'ADPDX_Archdiocesan_Assignment__c': {'Yes': True, 'No': ''}})
    .fillna('')
)

# Display a sample of the DataFrame to check the new structure
df_affiliations.sample(10)



,mbfc__Custom_Title__c,ADPDX_Archdiocesan_Assignment__c,Duty_Load__c,mbfc__Start_Date__c,mbfc__Completion_Date__c,Account_Ext_Id,mbfc__Person__r,mbfc__Context__r,mbfc__Use_Custom_Title__c
Record Number,,,,,,,,,
759,"Cedars-Medical Center, LA",True,,2019-05-15,2019-08-15,Organization_22,{'Adpdx_Migration_Id__c': '531'},{'Adpdx_Migration_Id__c': 'Organization_22'},True
303,Pastor,True,,2014-07-01,,Church_122,{'Adpdx_Migration_Id__c': '785'},{'Adpdx_Migration_Id__c': 'Church_122'},True
1569,Maintenance & Facilities Supervisor,,,,,Church_9,{'Adpdx_Migration_Id__c': '2117'},{'Adpdx_Migration_Id__c': 'Church_9'},True
2872,Pastor,True,,2022-07-01,,Church_123,{'Adpdx_Migration_Id__c': '838'},{'Adpdx_Migration_Id__c': 'Church_123'},True
876,Early Childhood Faith Formation Coordinator,,,,2023-01-01,Church_11,{'Adpdx_Migration_Id__c': '1655'},{'Adpdx_Migration_Id__c': 'Church_11'},True
291,Parochial Vicar,True,,,2019-02-28,Church_70,{'Adpdx_Migration_Id__c': '665'},{'Adpdx_Migration_Id__c': 'Church_70'},True
3505,Pasotral Assignment,True,Full Time,2024-01-01,,Organization_22,{'Adpdx_Migration_Id__c': '3019'},{'Adpdx_Migration_Id__c': 'Organization_22'},True
309,Pastor,True,,2016-07-01,2022-06-30,Church_128,{'Adpdx_Migration_Id__c': '599'},{'Adpdx_Migration_Id__c': 'Church_128'},True
2953,Parochial Vicar,True,,1995-08-01,2004-06-30,Church_80,{'Adpdx_Migration_Id__c': '1303'},{'Adpdx_Migration_Id__c': 'Church_80'},True


In [118]:
# @title Create ExternalID and Job Id
df_affiliations['Archdpdx_Migration_Id__c'] = df_affiliations.index

df_affiliations['Archdpdx_Job_Id__c'] = curr_job_id

df_affiliations.sample(5)

,mbfc__Custom_Title__c,ADPDX_Archdiocesan_Assignment__c,Duty_Load__c,mbfc__Start_Date__c,mbfc__Completion_Date__c,Account_Ext_Id,mbfc__Person__r,mbfc__Context__r,mbfc__Use_Custom_Title__c,Archdpdx_Migration_Id__c,Archdpdx_Job_Id__c
Record Number,,,,,,,,,,,
230,Pastor,True,,2016-07-01,2018-06-30,Church_16,{'Adpdx_Migration_Id__c': '583'},{'Adpdx_Migration_Id__c': 'Church_16'},True,230,49
2970,Pastor,True,,1988-07-01,1991-07-30,Church_12,{'Adpdx_Migration_Id__c': '1003'},{'Adpdx_Migration_Id__c': 'Church_12'},True,2970,49
1208,Office Administrator,,,,,Church_151,{'Adpdx_Migration_Id__c': '1988'},{'Adpdx_Migration_Id__c': 'Church_151'},True,1208,49
112,Deacon,True,,2005-11-05,2018-08-12,Church_90,{'Adpdx_Migration_Id__c': '191'},{'Adpdx_Migration_Id__c': 'Church_90'},True,112,49
3482,Main Office Administrative Assistant,,,2023-01-01,,School_15,{'Adpdx_Migration_Id__c': '3300'},{'Adpdx_Migration_Id__c': 'School_15'},True,3482,49


In [119]:
df_affiliations.to_csv('affiliations_staging.csv')

In [120]:
# @title Upsert Register Entry Records

bulk_data = []
for row in df_affiliations.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)



In [121]:
# Keep the batch <100 as I've been getting an exceptionCode: 'InvalidBatch', 'exceptionMessage': 'Records not processed'
affiliation_upsert = sf.bulk.mbfc__Placement__c.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
affiliation_upsert_results = pd.DataFrame(affiliation_upsert)

affiliation_upsert_results

SalesforceMalformedRequest: Malformed request https://adpdx--devpro.sandbox.my.salesforce.com/services/async/57.0/job/750Dx000007zgcRIAQ/batch/751Dx000009HBnYIAW/result. Response content: {'exceptionCode': 'InvalidBatch', 'exceptionMessage': 'Records not processed'}

In [ ]:
# @title Get Accounts and Contacts xref data from SF

# get SF Account
get_all_accounts = 'Select Archdpdx_Migration_Id__c, id, Name, RecordTypeId, Type, mbfc__Parish_Code__c, Job_Id__c from Account'

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')
df_sf_accounts


In [ ]:
# get SF Contacts
get_all_contacts = 'Select Archdpdx_Migration_Id__c, id, Name, Archdpdx_Job_Id__c from Contact'

# get list of records, add to dataframe
sf_contacts = sf.query(get_all_contacts)
df_sf_contacts = pd.DataFrame(sf_contacts['records'])
df_sf_contacts = df_sf_contacts.drop(columns = 'attributes')
df_sf_contacts

In [ ]:
# @title Map in SF Contact and Account IDs

# Post-Migration Manual Updates

1. Convert 'Offices' that are ADPDX Pastoral Centre offices into record type: 'Groups', and set their parentID to the Diocese (there are just 6 of these accounts).
1. Update the Religous Order records 'Religious Superior' lookup.
1. Set 'organization type' field value for each account in the 'organization' load: Offices, Newman Centres, Schools, Organizations
